In [14]:

# 基于迁移学习的两阶段方法 - 消融实验版本

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import copy
import os
import pandas as pd
from sklearn.metrics import mean_squared_error
import torch.nn.functional as F

# 导入数据加载器和评估函数
from dataloader import create_all_dataloaders, create_category_dataloaders, create_transfer_dataloaders, ChronosDataset
from calculate import calculate_metrics, print_metrics_table, calculate_uncertainty_metrics, calculate_category_pir, plot_uncertainty_comparison, plot_uncertainty_quality

# 设置随机种子以确保可重复性
torch.manual_seed(42)
np.random.seed(42)

# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ========================= 消融实验模型定义 =========================

class SimpleEncoder(nn.Module):
    """
    简单的LSTM编码器，用于替代MultiScaleEncoder和temporal_attention
    """
    def __init__(self, input_dim, hidden_dim, num_layers=2, dropout=0.3):
        super(SimpleEncoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # 使用标准LSTM作为特征提取器
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False  # 使用单向LSTM简化模型
        )
        
        # 特征投影层
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        # LSTM特征提取
        lstm_out, _ = self.lstm(x)
        
        # 特征投影
        projected_features = self.projection(lstm_out)
        
        return projected_features

class TimeSeriesEncoderAblation(nn.Module):
    """
    消融实验版本的时间序列编码器，移除了MultiScaleEncoder和temporal_attention
    """
    def __init__(self, input_dim, hidden_dim, num_domains=7, num_layers=2, dropout=0.3):
        super(TimeSeriesEncoderAblation, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_domains = num_domains
        
        # 使用简单编码器替代MultiScaleEncoder和temporal_attention
        self.simple_encoder = SimpleEncoder(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            dropout=dropout
        )
        
        # 保留贝叶斯域自适应模块
        self.domain_mu = nn.Parameter(torch.zeros(num_domains, hidden_dim))
        self.domain_logvar = nn.Parameter(torch.zeros(num_domains, hidden_dim))
        self.domain_importance = nn.Parameter(torch.ones(num_domains))
        
        # 域适应基础网络
        self.domain_adapter_base = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
    
    def bayesian_domain_adapt(self, features, domain_idx=None):
        # 贝叶斯域自适应方法（与原始实现相同）
        if domain_idx is not None:
            # 单域适应模式
            mu = self.domain_mu[domain_idx]
            logvar = self.domain_logvar[domain_idx]
            importance = F.softplus(self.domain_importance[domain_idx])
            
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            domain_params = mu + eps * std
            
            adapted_features = features * (domain_params * importance.unsqueeze(-1))
        else:
            # 混合域适应模式
            domain_weights = F.softmax(self.domain_importance, dim=0)
            
            mixed_mu = torch.sum(self.domain_mu * domain_weights.unsqueeze(1), dim=0)
            mixed_logvar = torch.sum(self.domain_logvar * domain_weights.unsqueeze(1), dim=0)
            
            std = torch.exp(0.5 * mixed_logvar)
            eps = torch.randn_like(std)
            domain_params = mixed_mu + eps * std
            
            adapted_features = features * domain_params
        
        return self.domain_adapter_base(adapted_features)
    
    def forward(self, x, domain_idx=None):
        batch_size, num_buildings, seq_len, _ = x.shape
        x_reshaped = x.view(batch_size * num_buildings, seq_len, self.input_dim)
        
        # 1. 简单特征提取
        encoded_features = self.simple_encoder(x_reshaped)
        
        # 2. 贝叶斯域适应
        adapted_features = []
        for t in range(seq_len):
            t_feat = encoded_features[:, t, :]
            t_adapted = self.bayesian_domain_adapt(t_feat, domain_idx)
            adapted_features.append(t_adapted)
        
        adapted_features = torch.stack(adapted_features, dim=1)
        
        # 恢复原始的批次和建筑物维度
        return adapted_features.view(batch_size, num_buildings, seq_len, self.hidden_dim)

class BiLSTMPredictorAblation(nn.Module):
    """
    消融实验版本的BiLSTM预测器，移除了注意力机制
    """
    def __init__(self, input_dim, hidden_dim, category_dim, forecast_horizon, num_buildings, num_layers=2, dropout=0.3):
        super(BiLSTMPredictorAblation, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.forecast_horizon = forecast_horizon
        self.num_buildings = num_buildings
        
        # 双向LSTM层
        self.bilstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # 类别特征编码层
        self.category_encoder = nn.Sequential(
            nn.Linear(category_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # 预测输出层（移除注意力机制）
        self.prediction_head = nn.Sequential(
            nn.Linear(hidden_dim*2 + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, forecast_horizon),
            nn.Sigmoid()
        )
    
    def forward(self, x, category):
        batch_size, num_buildings, seq_len, _ = x.shape
        
        # 编码类别特征
        category_embed = self.category_encoder(category)
        
        all_predictions = []
        for b in range(num_buildings):
            building_data = x[:, b, :, :]
            
            # 通过双向LSTM提取时序特征
            lstm_out, _ = self.bilstm(building_data)
            
            # 直接使用最后一个时间步的特征（移除注意力机制）
            final_hidden = lstm_out[:, -1, :]
            
            # 拼接时序特征和类别特征
            combined = torch.cat([final_hidden, category_embed], dim=1)
            
            # 生成预测结果
            pred = self.prediction_head(combined).unsqueeze(1)
            
            all_predictions.append(pred)
        
        return torch.cat(all_predictions, dim=1)

class AdaptiveBiLSTMAblation(nn.Module):
    """
    消融实验版本的AdaptiveBiLSTM，移除了MultiScaleEncoder和temporal_attention
    """
    def __init__(self, input_dim, hidden_dim, category_dim, forecast_horizon, num_buildings, num_domains=7, num_layers=2, dropout=0.3):
        super(AdaptiveBiLSTMAblation, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.forecast_horizon = forecast_horizon
        self.num_buildings = num_buildings
        self.num_layers = num_layers
        self.dropout = dropout
        
        # 使用消融实验版本的编码器
        self.time_series_encoder = TimeSeriesEncoderAblation(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            num_domains=num_domains,
            num_layers=num_layers,
            dropout=dropout
        )
        
        # 使用消融实验版本的预测器
        self.bilstm_predictor = BiLSTMPredictorAblation(
            input_dim=hidden_dim,
            hidden_dim=hidden_dim,
            category_dim=category_dim,
            forecast_horizon=forecast_horizon,
            num_buildings=num_buildings,
            num_layers=num_layers,
            dropout=dropout
        )
    
    def forward(self, x, category, domain_idx=None):
        # 提取时间特征并应用贝叶斯域自适应
        time_features = self.time_series_encoder(x, domain_idx)
        
        # 预测
        predictions = self.bilstm_predictor(time_features, category)
        
        return predictions

class DomainDiscriminator(nn.Module):
    def __init__(self, feature_dim, hidden_dim=64, dropout=0.3):
        super(DomainDiscriminator, self).__init__()
        self.feature_dim = feature_dim
        
        self.simple_model = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)  # 二分类，输出一个值
        )
        
    def forward(self, x):
        return self.simple_model(x)

# ========================= 模型参数设置 =========================

batch_size = 32
sequence_length = 24
forecast_horizon = 24
hidden_dim = 64
num_layers = 2
dropout = 0.2
learning_rate = 0.001
weight_decay = 1e-4
epochs = 15  # 源域预训练轮次
input_dim = 6
num_domains = 7  # 新增：域的数量  0是源域索引

# ========================= 辅助函数 =========================

def predict_with_uncertainty(model, inputs, category, domain_idx=None, mc_samples=100, device='cuda'):
    """
    使用MC Dropout进行不确定性估计 - 适配新的模型接口
    """
    model = model.to(device)
    inputs = inputs.to(device)
    category = category.to(device)
    model.train()  # 开启dropout以进行随机采样
    
    predictions = []
    for _ in range(mc_samples):
        # 修改: 使用新的模型接口
        outputs = model(inputs, category, domain_idx=domain_idx)  # 添加domain_idx参数
        
        if isinstance(outputs, tuple):
            outputs = outputs[0]  # 如果模型返回元组，取第一个元素
        
        predictions.append(outputs.detach())
    
    # 将预测堆叠为形状[mc_samples, batch, buildings, forecasts]
    try:
        stacked_preds = torch.stack(predictions, dim=0)
        
        # 计算平均值和标准差
        mean_pred = torch.mean(stacked_preds, dim=0)
        std_pred = torch.std(stacked_preds, dim=0)
        
        # 计算95%置信区间
        lower_bound = mean_pred - 1.96 * std_pred
        upper_bound = mean_pred + 1.96 * std_pred
        
        # 确保边界在有效范围内
        lower_bound = torch.clamp(lower_bound, 0, 1)
        upper_bound = torch.clamp(upper_bound, 0, 1)
        
        # 转换为CPU NumPy数组
        return (mean_pred.cpu().numpy(),
                lower_bound.cpu().numpy(),
                upper_bound.cpu().numpy(),
                std_pred.cpu().numpy())
    
    except Exception as e:
        print(f"处理MC采样结果时出错: {str(e)}")
        # 如果堆叠或其他操作失败，使用第一个样本作为预测
        try:
            mean_pred = predictions[0].cpu().numpy()
            std_pred = np.ones_like(mean_pred) * 0.1
            lower_bound = np.maximum(mean_pred - 1.96 * std_pred, 0)
            upper_bound = np.minimum(mean_pred + 1.96 * std_pred, 1)
            return mean_pred, lower_bound, upper_bound, std_pred
        except:
            # 最后的后备方案：返回零数组
            shape = list(inputs.shape)
            if len(shape) >= 3:
                shape[-1] = 1
            zeros = np.zeros(shape)
            return zeros, zeros, zeros, zeros


import torch
import numpy as np

def predict_with_uncertainty(model, inputs, category, domain_idx=None, mc_samples=100, device='cuda'):
    """
    使用MC Dropout进行不确定性估计 - 适配新的模型接口
    """
    model = model.to(device)
    inputs = inputs.to(device)
    category = category.to(device)
    model.train()  # 开启dropout以进行随机采样
    
    predictions = []
    for _ in range(mc_samples):
        # 修改: 使用新的模型接口
        outputs = model(inputs, category, domain_idx=domain_idx)  # 添加domain_idx参数
        
        if isinstance(outputs, tuple):
            outputs = outputs[0]  # 如果模型返回元组，取第一个元素
        
        predictions.append(outputs.detach())
    
    # 将预测堆叠为形状[mc_samples, batch, buildings, forecasts]
    try:
        stacked_preds = torch.stack(predictions, dim=0)
        
        # 计算平均值和标准差
        mean_pred = torch.mean(stacked_preds, dim=0)
        std_pred = torch.std(stacked_preds, dim=0)
        
        # 计算95%置信区间
        lower_bound = mean_pred - 1.96 * std_pred
        upper_bound = mean_pred + 1.96 * std_pred
        
        # 确保边界在有效范围内
        lower_bound = torch.clamp(lower_bound, 0, 1)
        upper_bound = torch.clamp(upper_bound, 0, 1)
        
        # 转换为CPU NumPy数组
        return (mean_pred.cpu().numpy(),
                lower_bound.cpu().numpy(),
                upper_bound.cpu().numpy(),
                std_pred.cpu().numpy())
    
    except Exception as e:
        print(f"处理MC采样结果时出错: {str(e)}")
        # 如果堆叠或其他操作失败，使用第一个样本作为预测
        try:
            mean_pred = predictions[0].cpu().numpy()
            std_pred = np.ones_like(mean_pred) * 0.1
            lower_bound = np.maximum(mean_pred - 1.96 * std_pred, 0)
            upper_bound = np.minimum(mean_pred + 1.96 * std_pred, 1)
            return mean_pred, lower_bound, upper_bound, std_pred
        except:
            # 最后的后备方案：返回零数组
            shape = list(inputs.shape)
            if len(shape) >= 3:
                shape[-1] = 1
            zeros = np.zeros(shape)
            return zeros, zeros, zeros, zeros
def calculate_transfer_metrics(source_features, target_features, source_outputs=None, target_outputs=None, baseline_predictions=None, targets=None):
    """
    计算迁移学习的综合评估指标
    
    Args:
        source_features: 源域特征 [batch_size, feature_dim]
        target_features: 目标域特征 [batch_size, feature_dim]
        source_outputs: 源域模型输出（可选）
        target_outputs: 目标域模型输出（可选）
        baseline_predictions: 基线模型预测（可选）
        targets: 真实目标值（可选，用于计算负迁移）
    """
    import traceback  # 导入traceback以便打印详细错误信息
    
    # 处理3D特征 - 将3D特征转换为2D
    if len(source_features.shape) == 3:  # [batch, seq_len, feature_dim]
        logging.info(f"检测到3D特征，进行展平: source_features.shape={source_features.shape}")
        # 方法2：平均所有时间步
        source_features = source_features.mean(dim=1)  # [batch, feature_dim]
        target_features = target_features.mean(dim=1)  # [batch, feature_dim]
        
        logging.info(f"展平后: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
    
    # 确保特征维度匹配
    if source_features.size(1) != target_features.size(1):
        logging.warning(f"特征维度不匹配: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
        return {
            'a_distance': 'N/A',
            'feature_alignment': 'N/A',
            'mmd': 'N/A',
            'sample_efficiency': None,
            'CC': 'N/A'  # 添加CC字段
        }
    
    # 检查特征数量是否过大，可能导致内存问题
    max_samples = 5000
    if source_features.size(0) > max_samples or target_features.size(0) > max_samples:
        logging.warning(f"特征数量过大，进行采样: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
        
        if source_features.size(0) > max_samples:
            indices = torch.randperm(source_features.size(0))[:max_samples]
            source_features = source_features[indices]
        
        if target_features.size(0) > max_samples:
            indices = torch.randperm(target_features.size(0))[:max_samples]
            target_features = target_features[indices]
        
        logging.info(f"采样后: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
    
    def calculate_mmd(x, y):
        """计算最大平均差异(MMD)"""
        try:
            x_kernel = torch.mm(x, x.t())
            y_kernel = torch.mm(y, y.t())
            xy_kernel = torch.mm(x, y.t())
            return x_kernel.mean() + y_kernel.mean() - 2 * xy_kernel.mean()
        except Exception as e:
            logging.error(f"计算MMD时出错: {str(e)}")
            return float('nan')
    
    def calculate_a_distance(source_features, target_features):
        """计算A-distance"""
        try:
            domain_classifier = nn.Sequential(
                nn.Linear(source_features.size(1), 50),
                nn.ReLU(),
                nn.Linear(50, 1)
            ).to(source_features.device)
            
            source_domain_labels = torch.ones(source_features.size(0), 1).to(source_features.device)
            target_domain_labels = torch.zeros(target_features.size(0), 1).to(source_features.device)
            
            features = torch.cat([source_features, target_features], dim=0)
            labels = torch.cat([source_domain_labels, target_domain_labels], dim=0)
            
            optimizer = torch.optim.Adam(domain_classifier.parameters())
            criterion = nn.BCEWithLogitsLoss()
            
            for _ in range(100):
                optimizer.zero_grad()
                preds = domain_classifier(features)
                loss = criterion(preds, labels)
                loss.backward()
                optimizer.step()
            
            with torch.no_grad():
                # 确保使用浮点数计算均值
                preds = torch.sigmoid(domain_classifier(features))
                predicted_labels = (preds > 0.5).float()
                error = (predicted_labels != labels).float().mean()
            
            return 2 * (1 - 2 * error)
        except Exception as e:
            logging.error(f"计算A-distance时出错: {str(e)}")
            traceback.print_exc()  # 打印详细错误信息
            return float('nan')
    
    def calculate_feature_alignment(source_features, target_features):
        """计算特征对齐质量"""
        try:
            source_norm = F.normalize(source_features, p=2, dim=1)
            target_norm = F.normalize(target_features, p=2, dim=1)
            similarity = torch.mm(source_norm, target_norm.t())
            return similarity.mean()
        except Exception as e:
            logging.error(f"计算特征对齐质量时出错: {str(e)}")
            return float('nan')
    
    # 添加计算相关系数(CC)的函数
    def calculate_correlation_coefficient(predictions, targets):
        """计算相关系数(CC)"""
        try:
            # 确保输入是numpy数组
            if isinstance(predictions, torch.Tensor):
                predictions = predictions.detach().cpu().numpy()
            if isinstance(targets, torch.Tensor):
                targets = targets.detach().cpu().numpy()
            
            # 展平数组
            predictions = np.array(predictions).flatten()
            targets = np.array(targets).flatten()
            
            # 确保长度相同
            min_length = min(len(predictions), len(targets))
            predictions = predictions[:min_length]
            targets = targets[:min_length]
            
            # 计算相关系数
            cc = np.corrcoef(predictions, targets)[0, 1]
            return cc
        except Exception as e:
            logging.error(f"计算相关系数时出错: {str(e)}")
            traceback.print_exc()
            return float('nan')

    try:
        metrics = {
            'a_distance': calculate_a_distance(source_features, target_features),
            'feature_alignment': calculate_feature_alignment(source_features, target_features),
            'mmd': calculate_mmd(source_features, target_features)
        }
        
        # 如果提供了目标域输出和真实目标值，计算相关系数
        if target_outputs is not None and targets is not None:
            metrics['CC'] = calculate_correlation_coefficient(target_outputs, targets)
        else:
            metrics['CC'] = float('nan')  # 如果没有提供必要数据，设置为NaN
            
    except Exception as e:
        logging.error(f"计算迁移学习指标时出错: {str(e)}")
        traceback.print_exc()  # 打印详细错误信息
        return {
            'a_distance': 'N/A',
            'feature_alignment': 'N/A',
            'mmd': 'N/A',
            'CC': 'N/A'  # 添加CC字段
        }
    
    # 如果提供了输出、基线预测和目标值，检测负迁移
    if target_outputs is not None and baseline_predictions is not None and targets is not None:
        try:
            # 确保所有输入都是张量并且形状匹配
            # 首先转换为numpy数组以便统一处理
            if isinstance(target_outputs, torch.Tensor):
                target_outputs = target_outputs.detach().cpu().numpy()
            if isinstance(baseline_predictions, torch.Tensor):
                baseline_predictions = baseline_predictions.detach().cpu().numpy()
            if isinstance(targets, torch.Tensor):
                targets = targets.detach().cpu().numpy()
            
            # 确保都是一维数组
            target_outputs = np.array(target_outputs).flatten()
            baseline_predictions = np.array(baseline_predictions).flatten()
            targets = np.array(targets).flatten()
            
            # 确保所有数组长度相同
            min_length = min(len(target_outputs), len(baseline_predictions), len(targets))
            target_outputs = target_outputs[:min_length]
            baseline_predictions = baseline_predictions[:min_length]
            targets = targets[:min_length]
            
            # 计算MSE损失
            target_loss = np.mean((target_outputs - targets) ** 2)
            baseline_loss = np.mean((baseline_predictions - targets) ** 2)
            
            # 打印调试信息
            logging.debug(f"target_loss: {target_loss}, type: {type(target_loss)}")
            logging.debug(f"baseline_loss: {baseline_loss}, type: {type(baseline_loss)}")
            
        except Exception as e:
            logging.error(f"计算负迁移指标时出错: {str(e)}")
            traceback.print_exc()  # 打印详细错误信息
    
    return metrics
def evaluate_model(model, test_loader, model_name="Model", baseline_model=None, device='cuda', domain_idx=None):
    """
    评估模型在测试集上的性能
    
    参数:
    - model: 要评估的模型（适配域自适应）
    - test_loader: 测试数据加载器
    - model_name: 模型名称
    - baseline_model: 基线模型(可选)，如果提供则用于计算PIR
    - device: 计算设备
    - domain_idx: 域索引(默认None，使用混合域)
    
    返回:
    - 评估指标字典
    """
    model.eval()
    all_predictions = []
    all_targets = []
    all_lower_bounds = []
    all_upper_bounds = []
    all_uncertainties = []
    # 用于迁移学习评估的特征收集
    source_features = []
    target_features = []
    source_outputs = []
    target_outputs = []
    
    # 计算标准预测和不确定性估计
    with torch.no_grad():
        for inputs, targets, _, category_onehot in test_loader:
            inputs = inputs.float().to(device)
            targets = targets.float().to(device)
            category_onehot = category_onehot.float().to(device)
            
            try:
                # 获取特征表示（用于迁移学习评估）
                if hasattr(model, 'time_series_encoder'):
                    try:
                        # 获取源域特征
                        source_feat = model.time_series_encoder(inputs, domain_idx=0)
                        source_features.append(source_feat.mean(dim=1))  # 平均池化时间维度
                        
                        # 获取目标域特征
                        target_feat = model.time_series_encoder(inputs, domain_idx=1)
                        target_features.append(target_feat.mean(dim=1))
                    except Exception as e:
                        print(f"提取域特征时出错: {str(e)}")
                        # 继续执行，不让特征提取错误影响主要评估
                
                # 使用MC Dropout估计不确定性 - 传递域索引参数
                mean_pred, lower_bound, upper_bound, uncertainty = predict_with_uncertainty(
                    model=model, 
                    inputs=inputs, 
                    category=category_onehot,  # 确保传递类别信息
                    domain_idx=domain_idx,     # 保持domain_idx参数
                    device=device, 
                    mc_samples=50
                )
                
                # 收集预测结果
                if isinstance(mean_pred, torch.Tensor):
                    mean_pred = mean_pred.cpu().numpy()
                
                # 尝试收集源域和目标域的输出
                try:
                    if hasattr(model, 'time_series_encoder'):
                        # 源域预测
                        source_outputs.append(mean_pred)
                        
                        # 使用目标域索引的预测
                        target_pred = model(inputs, category_onehot, domain_idx=domain_idx)
                        if isinstance(target_pred, torch.Tensor):
                            target_pred = target_pred.cpu().numpy()
                        target_outputs.append(target_pred)
                except Exception as e:
                    print(f"收集域输出时出错: {str(e)}")
                    # 继续执行，不让域输出收集错误影响主要评估
                
                # 收集预测、目标和不确定性信息
                all_predictions.append(mean_pred)
                all_targets.append(targets.cpu().numpy())
                all_lower_bounds.append(lower_bound)
                all_upper_bounds.append(upper_bound)
                all_uncertainties.append(uncertainty)
            except Exception as e:
                print(f"批次处理过程中发生错误: {str(e)}")
                print(f"跳过此批次")
                continue
    
    # 检查是否有有效预测
    if not all_predictions:
        print("警告: 没有收集到任何有效预测，无法评估模型")
        return {"Model": model_name, "Error": "无有效预测"}
    
    # 合并所有批次的预测和目标
    try:
        all_predictions = np.concatenate(all_predictions, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)
    except Exception as e:
        print(f"合并预测和目标时出错: {str(e)}")
        return {"Model": model_name, "Error": f"合并数据失败: {str(e)}"}
    
    # 检查不确定性相关的列表是否为空或合并是否成功
    if not all_lower_bounds or not all_upper_bounds or not all_uncertainties:
        print("警告: 不确定性估计列表为空，使用默认值")
        # 创建默认的不确定性估计
        all_lower_bounds = all_predictions * 0.9  # 简单地使用预测值的90%作为下界
        all_upper_bounds = all_predictions * 1.1  # 预测值的110%作为上界
        all_uncertainties = np.ones_like(all_predictions) * 0.1  # 默认不确定性为0.1
    else:
        # 合并所有批次的不确定性估计
        try:
            all_lower_bounds = np.concatenate(all_lower_bounds, axis=0)
            all_upper_bounds = np.concatenate(all_upper_bounds, axis=0)
            all_uncertainties = np.concatenate(all_uncertainties, axis=0)
        except Exception as e:
            print(f"合并不确定性估计时出错: {str(e)}")
            # 如果合并失败，使用默认值
            all_lower_bounds = all_predictions * 0.9
            all_upper_bounds = all_predictions * 1.1
            all_uncertainties = np.ones_like(all_predictions) * 0.1
    
    # 展平数组以便计算指标
    # 确保所有数组具有相同的形状
    all_predictions_flat = all_predictions.reshape(-1)
    all_targets_flat = all_targets.reshape(-1)
    
    # 确保不确定性相关的数组与预测和目标具有相同形状
    # 如果形状不同，可能需要广播或重采样
    if all_lower_bounds.size != all_predictions_flat.size:
        print(f"警告: 下界形状 {all_lower_bounds.shape} 与预测形状 {all_predictions.shape} 不一致")
        # 重塑或截断
        all_lower_bounds = all_lower_bounds.reshape(-1)[:all_predictions_flat.size]
        all_upper_bounds = all_upper_bounds.reshape(-1)[:all_predictions_flat.size]
        all_uncertainties = all_uncertainties.reshape(-1)[:all_predictions_flat.size]
    else:
        all_lower_bounds = all_lower_bounds.reshape(-1)
        all_upper_bounds = all_upper_bounds.reshape(-1)
        all_uncertainties = all_uncertainties.reshape(-1)
    
    # 打印形状信息用于调试
    print(f"形状信息:")
    print(f"预测: {all_predictions_flat.shape}")
    print(f"目标: {all_targets_flat.shape}")
    print(f"下界: {all_lower_bounds.shape}")
    print(f"上界: {all_upper_bounds.shape}")
    print(f"不确定性: {all_uncertainties.shape}")
    
    # 如果提供了基线模型，获取基线预测
    baseline_predictions = None
    if baseline_model is not None:
        baseline_model.eval()
        all_baseline_predictions = []
        
        with torch.no_grad():
            for inputs, _, _, category_onehot in test_loader:
                try:
                    inputs = inputs.float().to(device)
                    category_onehot = category_onehot.float().to(device)
                    
                    # 根据基线模型类型获取预测
                    if hasattr(baseline_model, 'predict'):
                        # 如果基线模型有predict方法
                        baseline_pred = baseline_model.predict(inputs)
                    else:
                        # 尝试直接获取预测值
                        baseline_pred = baseline_model(inputs, category_onehot)
                        # 处理返回元组的情况
                        if isinstance(baseline_pred, tuple):
                            baseline_pred = baseline_pred[0]
                    
                    # 转换为NumPy数组
                    if isinstance(baseline_pred, torch.Tensor):
                        baseline_pred = baseline_pred.cpu().numpy()
                    
                    all_baseline_predictions.append(baseline_pred)
                except Exception as e:
                    print(f"获取基线预测时出错: {str(e)}")
                    continue
        
        # 合并所有批次的基线预测
        if all_baseline_predictions:
            try:
                baseline_predictions = np.concatenate(all_baseline_predictions, axis=0).reshape(-1)
                
                # 确保基线预测具有相同的大小
                if baseline_predictions.size != all_predictions_flat.size:
                    print(f"警告: 基线预测形状 {baseline_predictions.shape} 与预测形状 {all_predictions_flat.shape} 不一致")
                    baseline_predictions = baseline_predictions[:all_predictions_flat.size]
            except Exception as e:
                print(f"处理基线预测时出错: {str(e)}")
                baseline_predictions = None
    
    # 改进的MAPE计算函数
    def calculate_improved_mape(y_true, y_pred, epsilon=0.01):
        """
        计算改进的MAPE，忽略接近零的值
        
        参数:
        - y_true: 真实值数组
        - y_pred: 预测值数组
        - epsilon: 阈值，避免除以接近零的值，对于0-1区间的数据，0.01是合理的
        
        返回:
        - MAPE值(百分比)
        """
        mask = np.abs(y_true) > epsilon
        if not np.any(mask):
            return float('nan')  # 如果没有有效值，返回NaN
        
        return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    
    # 计算评估指标，包含基线预测和置信区间
    try:
        metrics = calculate_metrics(
            predictions=all_predictions_flat, 
            real_values=all_targets_flat, 
            model_name=model_name, 
            baseline_predictions=baseline_predictions,
            lower_bounds=all_lower_bounds,
            upper_bounds=all_upper_bounds,
            confidence=0.95
        )
        
        # 添加平均不确定性
        metrics['avg_uncertainty'] = np.mean(all_uncertainties)
        
        # 使用改进的MAPE计算
        try:
            metrics['MAPE'] = calculate_improved_mape(all_targets_flat, all_predictions_flat, epsilon=0.01)
        except Exception as e:
            print(f"计算改进MAPE时出错: {str(e)}")
            # 保留原始MAPE或设置为NaN
            if 'MAPE' not in metrics:
                metrics['MAPE'] = float('nan')
    
        # 计算迁移学习指标 - 仅当所有必要的列表都非空时
        if (source_features and target_features and 
            source_outputs and target_outputs):
            try:
                # 处理特征数据
                source_features = torch.cat(source_features, dim=0).cpu()
                target_features = torch.cat(target_features, dim=0).cpu()
                source_outputs = np.concatenate(source_outputs, axis=0)
                target_outputs = np.concatenate(target_outputs, axis=0)
    
                # 计算迁移学习相关指标
                transfer_metrics = calculate_transfer_metrics(
                    source_features=source_features,
                    target_features=target_features,
                    source_outputs=source_outputs,
                    target_outputs=target_outputs,
                    baseline_predictions=baseline_predictions,
                    targets=all_targets_flat
                )
                # 更新指标字典
                metrics.update(transfer_metrics)
            except Exception as e:
                print(f"计算迁移学习指标时出错: {str(e)}")
                # 不要让这个错误影响整个评估流程
        else:
            print("警告: 迁移学习特征或输出列表为空，跳过迁移学习指标计算")
    
    except Exception as e:
        print(f"计算指标时出错: {str(e)}")
        # 创建一个基本的指标集作为回退
        from sklearn.metrics import mean_squared_error, r2_score
        rmse = np.sqrt(mean_squared_error(all_targets_flat, all_predictions_flat))
        r2 = r2_score(all_targets_flat, all_predictions_flat)
        
        metrics = {
            'Model': model_name,
            'RMSD': rmse,
            'R2': r2,
            'avg_uncertainty': np.mean(all_uncertainties)
        }
        
        # 尝试计算改进的MAPE
        try:
            metrics['MAPE'] = calculate_improved_mape(all_targets_flat, all_predictions_flat, epsilon=0.01)
        except Exception as e:
            print(f"计算改进MAPE时出错: {str(e)}")
            metrics['MAPE'] = float('nan')
    
# 打印详细的评估报告
    print(f"\n{model_name} 评估报告:")
    print(f"RMSD: {metrics.get('RMSD', 'N/A')}")
    print(f"MAPE: {metrics.get('MAPE', 'N/A')}%")
    print(f"R²: {metrics.get('R2', 'N/A')}")
    print(f"KL散度: {metrics.get('KL_divergence', 'N/A')}")
    
    # 打印迁移学习指标 - 安全处理格式化
    if any(key in metrics for key in ['a_distance', 'feature_alignment', 'mmd']):
        print("\n迁移学习评估:")
        
        # 安全处理a_distance
        a_distance = metrics.get('a_distance', 'N/A')
        if isinstance(a_distance, (int, float)) and not (isinstance(a_distance, float) and np.isnan(a_distance)):
            print(f"域间距离 (A-distance): {a_distance:.4f}")
        else:
            print(f"域间距离 (A-distance): {a_distance}")
        
        # 安全处理feature_alignment
        feature_alignment = metrics.get('feature_alignment', 'N/A')
        if isinstance(feature_alignment, (int, float)) and not (isinstance(feature_alignment, float) and np.isnan(feature_alignment)):
            print(f"特征对齐质量: {feature_alignment:.4f}")
        else:
            print(f"特征对齐质量: {feature_alignment}")
        
        # 安全处理mmd
        mmd = metrics.get('mmd', 'N/A')
        if isinstance(mmd, (int, float)) and not (isinstance(mmd, float) and np.isnan(mmd)):
            print(f"MMD: {mmd:.4f}")
        else:
            print(f"MMD: {mmd}")
        
  
    # 打印不确定性评估指标 - 安全处理
    if 'PICP' in metrics:
        print(f"\n不确定性评估:")
        
        picp = metrics.get('PICP', 'N/A')
        if isinstance(picp, (int, float)) and not (isinstance(picp, float) and np.isnan(picp)):
            print(f"预测区间覆盖率(PICP): {picp:.2f}% (目标95%)")
        else:
            print(f"预测区间覆盖率(PICP): {picp}% (目标95%)")
            
        nmpiw = metrics.get('NMPIW', 'N/A')
        if isinstance(nmpiw, (int, float)) and not (isinstance(nmpiw, float) and np.isnan(nmpiw)):
            print(f"平均预测区间宽度(NMPIW): {nmpiw:.4f}")
        else:
            print(f"平均预测区间宽度(NMPIW): {nmpiw}")
            
        calibration_error = metrics.get('calibration_error', 'N/A')
        if isinstance(calibration_error, (int, float)) and not (isinstance(calibration_error, float) and np.isnan(calibration_error)):
            print(f"校准误差: {calibration_error:.2f}%")
        else:
            print(f"校准误差: {calibration_error}")
            

            
        avg_uncertainty = metrics.get('avg_uncertainty', 'N/A')
        if isinstance(avg_uncertainty, (int, float)) and not (isinstance(avg_uncertainty, float) and np.isnan(avg_uncertainty)):
            print(f"平均不确定性(标准差): {avg_uncertainty:.4f}")
        else:
            print(f"平均不确定性(标准差): {avg_uncertainty}")
    
    if 'PIR' in metrics and metrics['PIR'] is not None:
        pir = metrics.get('PIR', 'N/A')
        if isinstance(pir, (int, float)) and not (isinstance(pir, float) and np.isnan(pir)):
            print(f"\n相比基线的改进率:")
            print(f"RMSD改进率(PIR): {pir:.2f}%")
        else:
            print(f"\n相比基线的改进率:")
            print(f"RMSD改进率(PIR): {pir}")

    
    # 尝试可视化一个样本的预测与置信区间
    # 尝试可视化一个样本的预测与置信区间
    try:
        sample_idx = 0  # 选择第一个样本进行可视化
        
        # 检查数据维度并适应相应的可视化方法
        if len(all_predictions.shape) == 3 and all_predictions.shape[0] > 0 and all_predictions.shape[1] > 0 and all_predictions.shape[2] > 0:
            # 3D数据: [batch, building, forecast]
            max_horizon = min(10, all_predictions.shape[2])  # 限制显示的预测长度
            
            plt.figure(figsize=(10, 6))
            try:
                plt.fill_between(
                    range(max_horizon),
                    all_lower_bounds.reshape(all_predictions.shape)[sample_idx, 0, :max_horizon],
                    all_upper_bounds.reshape(all_predictions.shape)[sample_idx, 0, :max_horizon],
                    alpha=0.3, color='blue', label='95% Confidence Interval'
                )
                plt.plot(range(max_horizon), all_predictions[sample_idx, 0, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, 0, :max_horizon], 'r-x', label='Ground Truth')
            except Exception as e:
                print(f"3D data visualization error: {str(e)}")
                # Try simplified visualization
                plt.plot(range(max_horizon), all_predictions[sample_idx, 0, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, 0, :max_horizon], 'r-x', label='Ground Truth')
        
        # 2D数据的情况
        elif len(all_predictions.shape) == 2 and all_predictions.shape[0] > 0 and all_predictions.shape[1] > 0:
            max_horizon = min(10, all_predictions.shape[1])
            
            plt.figure(figsize=(10, 6))
            try:
                plt.fill_between(
                    range(max_horizon),
                    all_lower_bounds.reshape(all_predictions.shape)[sample_idx, :max_horizon],
                    all_upper_bounds.reshape(all_predictions.shape)[sample_idx, :max_horizon],
                    alpha=0.3, color='blue', label='95% Confidence Interval'
                )
                plt.plot(range(max_horizon), all_predictions[sample_idx, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, :max_horizon], 'r-x', label='Ground Truth')
            except Exception as e:
                print(f"2D data visualization error: {str(e)}")
                # Try simplified visualization
                plt.plot(range(max_horizon), all_predictions[sample_idx, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, :max_horizon], 'r-x', label='Ground Truth')
        
        # 1D数据的情况
        elif len(all_predictions.shape) == 1 and all_predictions.shape[0] > 0:
            max_points = min(10, all_predictions.shape[0])
            
            plt.figure(figsize=(10, 6))
            try:
                plt.fill_between(
                    range(max_points),
                    all_lower_bounds[:max_points],
                    all_upper_bounds[:max_points],
                    alpha=0.3, color='blue', label='95% Confidence Interval'
                )
                plt.plot(range(max_points), all_predictions[:max_points], 'b-o', label='Predictions')
                plt.plot(range(max_points), all_targets[:max_points], 'r-x', label='Ground Truth')
            except Exception as e:
                print(f"1D data visualization error: {str(e)}")
                # Try the simplest visualization
                plt.plot(range(max_points), all_predictions[:max_points], 'b-o', label='Predictions')
                plt.plot(range(max_points), all_targets[:max_points], 'r-x', label='Ground Truth')
        
        else:
            print(f"Cannot visualize predictions: incompatible shape {all_predictions.shape}")
            return metrics
        
        plt.title(f'{model_name} Prediction Example\nPICP: {metrics.get("PICP", "N/A")}%')
        plt.xlabel('Prediction Time Step')
        plt.ylabel('Value')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Save the image, ensure the file name is valid
        safe_model_name = ''.join(c if c.isalnum() or c in ['-', '_'] else '_' for c in model_name)
        plt.savefig(f'prediction_sample_{safe_model_name}.png')
        plt.close()

    except Exception as e:
        print(f"创建可视化时出错: {str(e)}")
        print(f"数据形状: 预测={all_predictions.shape}, 目标={all_targets.shape}, 下界={all_lower_bounds.shape}, 上界={all_upper_bounds.shape}")
    
    return metrics

使用设备: cuda


In [15]:
from bilstm import BaselineBiLSTM
import os
import json
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import logging
from tqdm.auto import tqdm
import warnings
import copy
from torch import optim

# 配置日志：同时输出到文件和控制台，记录时间、等级和消息
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),  # 日志文件
        logging.StreamHandler()               # 控制台输出
    ]
)

# 忽略警告
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)



# 定义Huber损失函数
class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(HuberLoss, self).__init__()
        self.delta = delta
    
    def forward(self, predictions, targets):
        # 计算预测值与目标值之间的差异
        diff = predictions - targets
        abs_diff = torch.abs(diff)
        
        # 应用Huber损失公式
        loss = torch.where(
            abs_diff <= self.delta,
            0.5 * diff * diff,
            self.delta * (abs_diff - 0.5 * self.delta)
        )
        
        return torch.mean(loss)

# 计算损失的辅助函数
def calculate_loss(predictions, targets):
    """
    计算Huber损失
    
    参数:
    - predictions: 预测值
    - targets: 真实值
    
    返回:
    - loss: Huber损失值
    """
    # 可以直接使用PyTorch的内置HuberLoss (如果PyTorch版本≥1.9)
    # 或者使用自定义实现
    loss_fn = HuberLoss(delta=1.0)
    return loss_fn(predictions, targets)
def train_and_save_model(model, train_loader, val_loader, epochs, lr, weight_decay, 
                       model_name, save_dir='models', device='cuda', early_stopping_patience=5,
                       source_domain_idx=0, target_domain_idx=None):
    """
    训练模型并保存最佳模型，支持域自适应
    
    新增参数:
    - source_domain_idx: 源域索引，默认为0
    - target_domain_idx: 用于验证，默认为0，使用源域验证
    """
    os.makedirs(save_dir, exist_ok=True)
    model = model.to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    
    # 初始化记录器
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_epoch = 0
    patience_counter = 0
    
    # 保存模型配置信息（不同模型有不同的属性）
    model_config = {}
    
    # 尝试获取常见模型属性（如果存在的话）
    for attr in ['input_dim', 'hidden_dim', 'forecast_horizon', 'num_buildings', 'num_layers', 'dropout']:
        if hasattr(model, attr):
            model_config[attr] = getattr(model, attr)
    
    # 根据模型类型获取特定属性
    is_adaptive_model = hasattr(model, 'time_series_encoder')
    if is_adaptive_model:
        # 检查是否是消融实验模型
        if isinstance(model, AdaptiveBiLSTMAblation):
            model_config['model_type'] = 'AdaptiveBiLSTMAblation'  # 消融实验模型
        else:
            model_config['model_type'] = 'AdaptiveBiLSTM'  # 原始模型
            
        if hasattr(model.time_series_encoder, 'num_domains'):
            model_config['num_domains'] = model.time_series_encoder.num_domains
        model_config['source_domain_idx'] = source_domain_idx
        model_config['target_domain_idx'] = target_domain_idx

    
    # 保存最佳模型的信息
    best_model_info = {
        'state_dict': None,
        'optimizer_state': None,
        'epoch': 0,
        'train_loss': float('inf'),
        'val_loss': float('inf'),
        'metrics': None
    }

    print(f"开始训练 {model_name}...")

    for epoch in range(epochs):
        # 训练阶段
        model.train()
        epoch_train_loss = 0
        train_steps = 0
        
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Training]')
        for batch in progress_bar:
            inputs, targets, category, category_onehot = [
                x.float().to(device) if torch.is_tensor(x) else x for x in batch
            ]
            
            optimizer.zero_grad()
            
            predictions = model(inputs, category_onehot, domain_idx=source_domain_idx)
                
            loss = calculate_loss(predictions, targets)  # 使用Huber损失函数
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_train_loss += loss.item()
            train_steps += 1
            progress_bar.set_postfix({'train_loss': f'{loss.item():.4f}'})
        
        avg_train_loss = epoch_train_loss / train_steps
        train_losses.append(avg_train_loss)
        
        # 验证阶段
        model.eval()
        epoch_val_loss = 0
        val_steps = 0
        
        with torch.no_grad():
            progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Validation]')
            for batch in progress_bar:
                inputs, targets, category, category_onehot = [
                    x.float().to(device) if torch.is_tensor(x) else x for x in batch
                ]
                    # 自适应模型验证也使用源域
                predictions = model(inputs, category_onehot, domain_idx=source_domain_idx)

                    
                loss = calculate_loss(predictions, targets)
                
                epoch_val_loss += loss.item()
                val_steps += 1
                progress_bar.set_postfix({'val_loss': f'{loss.item():.4f}'})
        
        avg_val_loss = epoch_val_loss / val_steps
        val_losses.append(avg_val_loss)
        
        # 学习率调整
        scheduler.step(avg_val_loss)
        
        # 计算当前模型的评估指标
        # 使用修改后的simple_evaluate_model，支持域自适应
        current_metrics = simple_evaluate_model(
            model, val_loader, f"{model_name}_epoch_{epoch+1}", 
            device=device, domain_idx=target_domain_idx if is_adaptive_model else None
        )
        
        # 检查是否是最佳模型
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_epoch = epoch
            patience_counter = 0
            
            # 更新最佳模型信息
            best_model_info = {
                'state_dict': copy.deepcopy(model.state_dict()),
                'optimizer_state': copy.deepcopy(optimizer.state_dict()),
                'epoch': epoch + 1,
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'metrics': current_metrics,
                'hyperparameters': {
                    'lr': lr,
                    'weight_decay': weight_decay,
                    'epochs': epochs,
                    'best_epoch': epoch + 1,
                    'model_config': model_config,
                    'model_type': type(model).__name__,
                    'source_domain_idx': source_domain_idx if is_adaptive_model else None,
                    'target_domain_idx': target_domain_idx if is_adaptive_model else None
                }
            }
            
            # 保存最佳模型检查点
            checkpoint_path = os.path.join(save_dir, f'{model_name}_best.pth')
            torch.save(best_model_info, checkpoint_path)
            print(f"✅ 保存最佳模型 (epoch {epoch+1}), 验证损失: {avg_val_loss:.4f}")
        else:
            patience_counter += 1
        
        # 打印当前epoch的训练信息
        print(
            f"Epoch {epoch+1}/{epochs} - "
            f"Train Loss: {avg_train_loss:.4f}, "
            f"Val Loss: {avg_val_loss:.4f}, "
            f"RMSD: {current_metrics['RMSD']:.4f}, "
            f"R²: {current_metrics['R2']:.4f}, "
            f"Best Val Loss: {best_val_loss:.4f} (Epoch {best_epoch+1}), "
            f"LR: {optimizer.param_groups[0]['lr']:.6f}"
        )
        
        # 早停检查
        if patience_counter >= early_stopping_patience:
            print(f"Early stopping triggered after epoch {epoch+1}")
            break
    
    
    # 训练结束后，保存训练历史
    history = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'best_epoch': best_epoch + 1,
        'best_val_loss': best_val_loss
    }
    
    # 保存训练历史
    history_path = os.path.join(save_dir, f'{model_name}_training_history.json')
    with open(history_path, 'w') as f:
        # 将列表转换为可序列化格式
        serializable_history = {
            'train_losses': [float(loss) for loss in train_losses],
            'val_losses': [float(loss) for loss in val_losses],
            'best_epoch': best_epoch + 1,
            'best_val_loss': float(best_val_loss)
        }
        json.dump(serializable_history, f, indent=4)
    
    # 绘制训练曲线
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.axvline(x=best_epoch, color='r', linestyle='--', label=f'Best Model (Epoch {best_epoch+1})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{model_name} Training History')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(save_dir, f'{model_name}_training_curve.png'))
    plt.close()
    
    # 恢复最佳模型状态
    model.load_state_dict(best_model_info['state_dict'])
    
    return model, best_model_info, history

In [16]:
# 修改简化评估函数，支持域自适应
def simple_evaluate_model(model, test_loader, model_name="Model", device='cuda', domain_idx=None):
    """
    训练中使用的简化评估函数，支持域自适应
    
    参数:
    - model: 要评估的模型
    - test_loader: 测试数据加载器
    - model_name: 模型名称
    - device: 计算设备
    - domain_idx: 域索引(None表示使用混合域)
    
    返回:
    - 简化的评估指标字典
    """
    from sklearn.metrics import mean_squared_error, r2_score
    
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets, _, category_onehot in test_loader:
            inputs = inputs.float().to(device)
            targets = targets.float().to(device)
            category_onehot = category_onehot.float().to(device)
            predictions = model(inputs, category_onehot, domain_idx=0)

            
            # 收集预测和目标值
            pred_values = predictions.cpu().numpy()
            target_values = targets.cpu().numpy()
            
            all_preds.append(pred_values)
            all_targets.append(target_values)
    
    # 后续代码保持不变...
    # 合并所有批次的预测和目标
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    # 展平数组以便计算指标
    all_preds = all_preds.reshape(-1)
    all_targets = all_targets.reshape(-1)
    
    # 计算基本指标
    mse = mean_squared_error(all_targets, all_preds)
    rmsd = np.sqrt(mse)
    r2 = r2_score(all_targets, all_preds)
    
    # 计算MAPE（排除零值）
    mask = np.abs(all_targets) > 1e-6
    mape = np.mean(np.abs((all_targets[mask] - all_preds[mask]) / all_targets[mask])) * 100 if np.any(mask) else np.nan
    
    # 计算CC (相关系数)
    cc = np.corrcoef(all_preds, all_targets)[0, 1]
    
    return {
        "Model": model_name,
        "RMSD": rmsd,
        "R2": r2,
        "MAPE": mape,
        "CC": cc
    }

# 创建保存目录
os.makedirs('models', exist_ok=True)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 加载数据
print("加载所有类别的训练数据...")
train_loader, test_loader, categories = create_all_dataloaders(
    batch_size=batch_size, 
    sequence_length=sequence_length, 
    forecast_horizon=forecast_horizon
)
from torch.utils.data import Subset, DataLoader, random_split

# 从train_loader中获取完整训练建筑数据集
all_train_dataset = train_loader.dataset  # 这是ConcatDataset

# 设定划分比例（如80%训练，20%验证）
train_ratio = 0.8
val_ratio = 0.2
total_len = len(all_train_dataset)
train_len = int(total_len * train_ratio)
val_len = total_len - train_len

# 使用random_split划分
train_subset, val_subset = random_split(all_train_dataset, [train_len, val_len])

# 构建新的dataloader
batch_size = train_loader.batch_size
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# 获取数据维度信息
for inputs, targets, category, category_onehot in train_loader:
    print(f"输入形状: {inputs.shape}, 目标形状: {targets.shape}, 类别形状: {category_onehot.shape}")
    input_dim = inputs.shape[-1]
    num_buildings = inputs.shape[1]
    category_dim = category_onehot.shape[-1]
    break
# 创建域自适应模型（消融实验版本）
print("创建和训练AdaptiveBiLSTMAblation模型...")
num_domains = 7  # 设置为7：1源域和6目标域
source_domain_idx = 0  # 源域索引
target_domain_idx = 0  # 评估时使用的仍然是源域

# 创建AdaptiveBiLSTMAblation模型（替换为消融实验版本）
source_model = AdaptiveBiLSTMAblation(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    category_dim=category_dim,
    forecast_horizon=forecast_horizon,
    num_buildings=num_buildings,
    num_domains=num_domains,  # 新增参数，指定域数量
    num_layers=num_layers,
    dropout=dropout
)

# 训练源域模型，使用修改后的训练函数
trained_source_model, source_best_info, source_history = train_and_save_model(
    model=source_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs,
    lr=learning_rate,
    weight_decay=weight_decay,
    model_name='ablation_source_huber',  # 修改模型名称以区分
    save_dir='models',
    device=device,
    source_domain_idx=source_domain_idx,  # 指定源域索引
    target_domain_idx=target_domain_idx   # 指定目标域索引
)

# 使用改进的完整评估函数对最终模型进行评估
print("\n评估消融实验模型性能...")
source_metrics = evaluate_model(
    model=trained_source_model, 
    test_loader=val_loader,
    model_name="Ablation_Source_Huber_Model",  # 修改模型名称以区分
    device=device,
    domain_idx=source_domain_idx  # 评估时使用源域
)

# 提取重要的评估指标
print("\n消融实验模型评估指标:")
print(f"MAPE(%): {source_metrics['MAPE']:.2f}")
print(f"RMSD: {source_metrics['RMSD']:.4f}")
print(f"R²: {source_metrics['R2']:.4f}")

def convert_to_serializable(obj):
    """将对象转换为JSON可序列化的格式"""
    if isinstance(obj, torch.Tensor):
        # 将张量转换为Python原生类型
        obj = obj.detach().cpu().numpy()
        if obj.size == 1:
            return float(obj.item())  # 单个值转为float
        return obj.tolist()  # 数组转为列表
    elif isinstance(obj, np.ndarray):
        # 将NumPy数组转换为列表
        return obj.tolist()
    elif isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
        # 将NumPy标量转换为Python标量
        return float(obj) if np.issubdtype(obj.dtype, np.floating) else int(obj)
    elif isinstance(obj, dict):
        # 递归处理字典
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        # 递归处理列表
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, (float, int, str, bool, type(None))):
        # 这些类型已经是JSON可序列化的
        return obj
    else:
        # 其他类型，尝试转换为字符串
        try:
            return str(obj)
        except:
            return "Non-serializable object"

# 打印源域的不确定性评估结果
if 'PICP' in source_metrics:
    print("\n消融实验模型不确定性评估:")
    print(f"预测区间覆盖率(PICP): {source_metrics['PICP']:.2f}% (目标95%)")
    print(f"校准误差: {source_metrics['calibration_error']:.2f}%")
    print(f"平均区间宽度(NMPIW): {source_metrics['NMPIW']:.4f}")
    print(f"不确定性质量分数(UQS): {source_metrics['UQS']:.4f}")

# 保存最终评估结果
metrics_path = os.path.join('models', 'ablation_model_evaluation_metrics.json')
with open(metrics_path, 'w') as f:
    all_metrics = {
        'source_domain': {k: convert_to_serializable(v) 
                         for k, v in source_metrics.items() if k != 'Model'},
        'model_info': convert_to_serializable({
            'best_epoch': source_best_info['epoch'],
            'num_domains': num_domains,
            'model_type': 'AdaptiveBiLSTMAblation'  # 修改模型类型
        })
    }
    
    json.dump(all_metrics, f, indent=4)

print("消融实验模型训练完成！")

使用设备: cuda
加载所有类别的训练数据...
加载天气数据: Rat_mild_train.csv, 形状: (2184, 5)
加载天气数据: Rat_mild_test.csv, 形状: (6600, 5)
加载天气数据: Hog_mild_train.csv, 形状: (2184, 5)
加载天气数据: Hog_mild_test.csv, 形状: (6600, 5)
加载天气数据: Robin_mild_train.csv, 形状: (2184, 5)
加载天气数据: Robin_mild_test.csv, 形状: (6600, 5)
加载天气数据: Wolf_mild_train.csv, 形状: (2184, 5)
加载天气数据: Wolf_mild_test.csv, 形状: (6600, 5)
加载天气数据: Eagle_mild_train.csv, 形状: (2184, 5)
加载天气数据: Eagle_mild_test.csv, 形状: (6600, 5)
加载天气数据: Gator_mild_train.csv, 形状: (2184, 5)
加载天气数据: Gator_mild_test.csv, 形状: (6600, 5)
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 DO 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 DO 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 HO 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 HO 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 LI 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 LI 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 OF 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 OF 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 UL

Epoch 1/15 [Training]:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 1/15 [Validation]:   0%|          | 0/67 [00:00<?, ?it/s]

✅ 保存最佳模型 (epoch 1), 验证损失: 0.0098
Epoch 1/15 - Train Loss: 0.0147, Val Loss: 0.0098, RMSD: 0.1394, R²: 0.4708, Best Val Loss: 0.0098 (Epoch 1), LR: 0.001000


Epoch 2/15 [Training]:   0%|          | 0/267 [00:00<?, ?it/s]

...

Epoch 15/15 [Training]:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 15/15 [Validation]:   0%|          | 0/67 [00:00<?, ?it/s]

Epoch 15/15 - Train Loss: 0.0037, Val Loss: 0.0035, RMSD: 0.0832, R²: 0.8116, Best Val Loss: 0.0034 (Epoch 13), LR: 0.001000

评估消融实验模型性能...


2025-06-20 20:09:43,855 - INFO - 检测到3D特征，进行展平: source_features.shape=torch.Size([2136, 24, 64])
2025-06-20 20:09:43,857 - INFO - 展平后: source_features.shape=torch.Size([2136, 64]), target_features.shape=torch.Size([2136, 64])


形状信息:
预测: (256320,)
目标: (256320,)
下界: (256320,)
上界: (256320,)
不确定性: (256320,)

Ablation_Source_Huber_Model 评估报告:
RMSD: 0.0819674368563001
MAPE: 15.075156211853027%
R²: 0.8169643878936768
KL散度: N/A

迁移学习评估:
域间距离 (A-distance): 1.3501873016357422
特征对齐质量: 0.7236200571060181
MMD: 0.7079057693481445

不确定性评估:
预测区间覆盖率(PICP): 65.82% (目标95%)
平均预测区间宽度(NMPIW): 0.11640872061252594
校准误差: 29.18%
平均不确定性(标准差): 0.02970263920724392

消融实验模型评估指标:
MAPE(%): 15.08
RMSD: 0.0820
R²: 0.8170

消融实验模型不确定性评估:
预测区间覆盖率(PICP): 65.82% (目标95%)
校准误差: 29.18%
平均区间宽度(NMPIW): 0.1164
不确定性质量分数(UQS): 0.4781
消融实验模型训练完成！


In [17]:
import traceback
import logging
from tqdm.auto import tqdm
import warnings
import torch.nn.functional as F
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Function
from datetime import datetime

# 梯度反转层 - 域对抗训练的核心组件
class GradientReversalFunction(Function):
    """
    梯度反转层 - 在反向传播时反转梯度方向，实现域对抗训练
    前向传播：直接传递输入
    反向传播：将梯度乘以负的alpha值，实现梯度反转
    """
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha  # 存储alpha参数用于反向传播
        return x.view_as(x)  # 前向传播不改变输入
    
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None  # 反向传播时反转梯度方向

def grad_reverse(x, alpha=1.0):
    """梯度反转函数的包装，便于调用"""
    return GradientReversalFunction.apply(x, alpha)

def domain_adversarial_training_step(
    source_features, 
    target_features, 
    domain_discriminator, 
    optimizer_disc, 
    epoch, 
    total_epochs,
    device,
    projection_layer=None,
    target_domain_idx=1  # 目标域索引（仅用于数据标识，不影响二分类标签）
):
    """
    执行单个域对抗训练步骤（二分类逻辑，源域vs目标域）
    
    Args:
        target_domain_idx: 目标域的索引值（仅用于数据层面的标识，标签统一为1）
    """
    try:
        # 打印输入特征形状用于调试
        logging.debug(f"Source features shape: {source_features.shape}")
        logging.debug(f"Target features shape: {target_features.shape}")
        
        batch_size = source_features.size(0)
        
        # 确保特征维度匹配
        if source_features.size(-1) != target_features.size(-1):
            raise ValueError(f"Feature dimensions don't match")
        
        # 处理4D特征格式
        if len(source_features.shape) == 4:
            source_features = source_features.reshape(-1, source_features.shape[-2], source_features.shape[-1])
            target_features = target_features.reshape(-1, target_features.shape[-2], target_features.shape[-1])
            batch_size = source_features.size(0)
        
        # 分离特征以避免重复反向传播
        source_features = source_features.detach()
        target_features = target_features.detach()
        
        # 直接使用原始特征而非多尺度特征
        source_processed = source_features
        target_processed = target_features
        
        # 准备二分类标签（源域=0，目标域=1）
        source_domain_labels = torch.zeros(batch_size, 1).to(device)
        target_domain_labels = torch.ones(batch_size, 1).to(device)
        
        # 特征降维处理
        if len(source_processed.shape) == 3:  # [batch, seq_len, hidden_dim]
            source_processed = source_processed.mean(dim=1)
            target_processed = target_processed.mean(dim=1)
        elif len(source_processed.shape) == 4:
            source_processed = source_processed.mean(dim=(1, 2))
            target_processed = target_processed.mean(dim=(1, 2))
        
        # 维度匹配处理
        expected_dim = domain_discriminator.feature_dim
        if source_processed.size(-1) != expected_dim:
            if projection_layer is not None:
                source_processed = projection_layer(source_processed)
                target_processed = projection_layer(target_processed)
            else:
                if not hasattr(domain_adversarial_training_step, 'projection_layer'):
                    domain_adversarial_training_step.projection_layer = nn.Linear(
                        source_processed.size(-1), expected_dim
                    ).to(device)
                source_processed = domain_adversarial_training_step.projection_layer(source_processed)
                target_processed = domain_adversarial_training_step.projection_layer(target_processed)
        
        # 连接特征和标签
        features = torch.cat([source_processed, target_processed], dim=0)
        domain_labels = torch.cat([source_domain_labels, target_domain_labels], dim=0)
        
        # 梯度反转参数
        grad_reverse_strength = 2. / (1. + np.exp(-10 * epoch / total_epochs)) - 1
        reversed_features = grad_reverse(features, grad_reverse_strength)
        
        # 域判别器预测
        domain_preds = domain_discriminator.simple_model(reversed_features)
        
        # 计算二分类损失
        domain_loss = F.binary_cross_entropy_with_logits(domain_preds, domain_labels)
        
        # 更新判别器
        optimizer_disc.zero_grad()
        domain_loss.backward(retain_graph=True)
        optimizer_disc.step()
        
        # 分离预测结果
        source_domain_preds = domain_preds[:batch_size]
        target_domain_preds = domain_preds[batch_size:]
        
        return domain_loss, source_domain_preds, target_domain_preds
        
    except Exception as e:
        logging.warning(f"批次处理出错: {str(e)}")
        traceback.print_exc()
        return (
            torch.tensor(0.0, device=device),
            torch.zeros(batch_size, 1, device=device),
            torch.zeros(batch_size, 1, device=device)
        )

# 自适应λ调度器 - 动态调整域对抗训练强度
def adaptive_lambda_scheduler(epoch, epochs, source_loss, target_loss, domain_loss, lambda_domain):
    """
    基于训练进度、任务损失和域判别损失动态调整梯度反转参数λ
    
    Args:
        epoch: 当前训练轮次
        epochs: 总训练轮次
        source_loss: 源域任务损失
        target_loss: 目标域任务损失
        domain_loss: 域判别损失
        lambda_domain: 基础λ值
    
    Returns:
        float: 调整后的λ值
    """
    # 1. 基于训练进度的基础调整
    progress = epoch / epochs
    
    # 训练初期：较小的λ值，专注于任务学习
    if progress < 0.3:
        base_lambda = max(0.001, lambda_domain * 0.01)
    # 训练中期：中等λ值，平衡任务学习和域适应
    elif progress < 0.7:
        base_lambda = max(0.005, lambda_domain * 0.05)
    # 训练后期：较大λ值，加强域适应
    else:
        base_lambda = max(0.01, lambda_domain * 0.1)
    
    # 2. 基于源域和目标域任务损失比例的调整
    task_ratio = target_loss / (source_loss + 1e-10)
    
    # 如果目标域损失远大于源域，减小λ以专注于任务学习
    if task_ratio > 2.0:
        adjust_factor = 0.5
    # 如果目标域损失远小于源域，增大λ以加强域适应
    elif task_ratio < 0.5:
        adjust_factor = 2.0
    else:
        adjust_factor = 1.0
    
    # 3. 基于域判别器性能的调整
    # 如果判别器损失接近0.693(log(2))，说明判别器无法区分域，减小λ
    if abs(domain_loss - 0.693) < 0.1:
        domain_factor = 0.8
    # 如果判别器损失太小，说明判别器过强，增大λ
    elif domain_loss < 0.3:
        domain_factor = 1.5
    else:
        domain_factor = 1.0
    
    # 计算最终λ值并限制在合理范围内
    final_lambda = base_lambda * adjust_factor * domain_factor
    return min(max(final_lambda, 0.001), 0.1)  # 限制范围 [0.001, 0.1]
import torch
from torch.autograd import Function
def adapt_to_target_domain_ablation(source_model, source_loader, target_loader, epochs=20, lr=0.001, 
                         device='cuda', lambda_domain=0.4, early_stopping_patience=3,
                         source_domain_idx=0, target_domain_idx=None):
    """
    将源域模型适应到目标域的改进版本，适用于AdaptiveBiLSTMAblation模型（消融实验版本）
    
    参数:
    - source_model: 预训练的源域模型（已在源域完成基础训练）
    - source_loader: 源域数据加载器（用于域对抗训练中的源域数据）
    - target_loader: 目标域数据加载器（用于域对抗训练和模型评估）
    - epochs: 迁移训练轮数（控制训练迭代次数）
    - lr: 主模型学习率（优化器初始学习率）
    - device: 计算设备（'cuda'或'cpu'）
    - lambda_domain: 域对抗损失基础权重（用于动态调度的初始值）
    - early_stopping_patience: 早停耐心值（连续多少轮未改进则停止训练）
    - source_domain_idx: 源域索引（数据集中源域的唯一标识）
    - target_domain_idx: 目标域索引（数据集中目标域的唯一标识）
    """
    
    # -------------------------- 日志与警告配置 --------------------------
    logging.basicConfig(
        level=logging.INFO,                  # 日志级别：仅记录INFO及以上信息
        format='%(asctime)s - %(levelname)s - %(message)s',  # 日志格式：时间-级别-消息
        handlers=[
            logging.FileHandler('transfer_learning_ablation.log'),  # 修改日志文件名以区分
            logging.StreamHandler()                        # 日志输出到控制台
        ]
    )
    
    # 忽略特定类型警告（避免无关警告干扰训练日志）
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    
    # -------------------------- 模型与组件初始化 --------------------------
    # 深拷贝源模型，避免修改原始模型
    model = copy.deepcopy(source_model).to(device)
    
    # 检测模型类型（是否为自适应模型，含时间序列编码器）
    is_adaptive_model = hasattr(model, 'time_series_encoder')
    encoder_name = 'time_series_encoder' if is_adaptive_model else 'chronos_encoder'
    encoder = getattr(model, encoder_name)  # 获取模型的编码器组件
    
    # -------------------------- 确定特征维度（关键预处理） --------------------------
    try:
        logging.info("获取样本以确定正确的特征维度...")
        # 从源域数据加载器中获取一个批次样本
        sample_source_batch = next(iter(source_loader))
        sample_source_inputs = sample_source_batch[0].float().to(device)  # 输入数据
        
        # 无梯度环境下提取特征（避免计算不必要的梯度）
        with torch.no_grad():
            # 使用源域索引提取特征（确保特征与源域对应）
            sample_source_features = model.time_series_encoder(sample_source_inputs, domain_idx=source_domain_idx)
        
        # 处理不同维度的特征张量（兼容3D和4D输入）
        if len(sample_source_features.shape) == 3:  # [batch, seq_len, hidden_dim]
            # 在时间维度上取平均，转为2D特征 [batch, hidden_dim]
            sample_source_features = sample_source_features.mean(dim=1)
        elif len(sample_source_features.shape) == 4:  # [batch, buildings, seq_len, hidden_dim]
            # 重塑并在时间维度上取平均，转为 [batch*buildings, hidden_dim]
            b, n, s, h = sample_source_features.shape
            sample_source_features = sample_source_features.reshape(b*n, s, h).mean(dim=1)
        
        # 获取特征维度（用于初始化域判别器）
        correct_feature_dim = sample_source_features.size(-1)
        logging.info(f"确定正确的特征维度: {correct_feature_dim}")
        
        # 创建域判别器（输入维度为提取的特征维度）
        domain_discriminator = DomainDiscriminator(
            feature_dim=correct_feature_dim,
            hidden_dim=64,
            dropout=0.3
        ).to(device)
    except Exception as e:
        logging.warning(f"特征维度确定失败: {str(e)}")
        # 使用模型编码器的隐藏维度作为默认值（ fallback 机制）
        domain_discriminator = DomainDiscriminator(
            feature_dim=encoder.hidden_dim,
            hidden_dim=64,
            dropout=0.3
        ).to(device)
    
    # -------------------------- 优化器与调度器设置 --------------------------
    # 主模型优化器（使用AdamW优化器，带权重衰减）
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    # 域判别器优化器（学习率为主模型的一半，避免判别器过强）
    optimizer_disc = optim.AdamW(domain_discriminator.parameters(), lr=lr*0.5, weight_decay=0.02)
    
    # 学习率调度器（余弦退火+重启，避免学习率过早收敛）
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=5, T_mult=2, eta_min=lr*0.1  # 初始周期5轮，最小学习率为初始的10%
    )
    scheduler_disc = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer_disc, T_0=5, T_mult=2, eta_min=lr*0.1
    )
    
    # 混合精度训练支持（自动缩放梯度，提升GPU效率）
    scaler = torch.amp.GradScaler() if torch.cuda.is_available() else None
    
    # -------------------------- 损失函数定义 --------------------------
    huber_loss_fn = HuberLoss(delta=1.0)  # 任务损失使用Huber损失（对异常值鲁棒）
    domain_criterion = nn.BCEWithLogitsLoss()  # 域对抗损失使用二分类交叉熵
    
    # -------------------------- 早停与模型保存配置 --------------------------
    best_rmse = float('inf')  # 最佳RMSE值（越低越好）
    patience_counter = 0  # 早停计数器
    best_model_state = None  # 最佳模型参数
    
    # 训练历史记录（用于分析训练过程）
    history = {'total_loss': [], 'task_loss': [], 'domain_loss': [], 'rmse': []}
    
    # 梯度裁剪阈值（防止梯度爆炸）
    max_grad_norm = 1.0
    
    logging.info("开始消融实验域自适应迁移学习训练...")
    
    # -------------------------- 域对抗组件初始化 --------------------------

    # 全局投影层（将特征投影到判别器所需维度）
    logging.info("创建全局特征投影层...")
    projection_layer = nn.Linear(model.hidden_dim, 64).to(device)  # 使用模型的hidden_dim
    optimizer_proj = optim.Adam(projection_layer.parameters(), lr=lr)  # 投影层单独优化器
    
    # -------------------------- 主训练循环 --------------------------
    try:
        # 初始化损失统计变量（用于动态调整λ）
        epoch_source_losses = []  # 源域任务损失列表
        epoch_target_losses = []  # 目标域任务损失列表
        epoch_domain_losses = []   # 域对抗损失列表
        
        for epoch in range(epochs):
            model.train()          # 设置模型为训练模式
            domain_discriminator.train()

            
            # ---------------- 动态调整λ参数（关键逻辑） ----------------
            if epoch > 0 and epoch_source_losses and epoch_target_losses and epoch_domain_losses:
                # 计算上一epoch的平均损失（用于自适应调度）
                avg_source_loss = sum(epoch_source_losses) / len(epoch_source_losses)
                avg_target_loss = sum(epoch_target_losses) / len(epoch_target_losses)
                avg_domain_loss = sum(epoch_domain_losses) / len(epoch_domain_losses)
                
                # 调用自适应调度函数，根据损失和进度计算当前λ
                current_lambda = adaptive_lambda_scheduler(
                    epoch-1, epochs,       # 当前轮次（从0开始）和总轮次
                    avg_source_loss, avg_target_loss, avg_domain_loss,  # 平均损失
                    lambda_domain          # 基础权重
                )
            else:
                # 第一个epoch使用初始值（训练初期λ较小，优先学习任务）
                current_lambda = max(0.001, lambda_domain * 0.01)
            
            # 重置当前epoch的损失统计
            epoch_stats = {'total_loss': 0, 'task_loss': 0, 'domain_loss': 0}
            epoch_source_losses = []
            epoch_target_losses = []
            epoch_domain_losses = []
            
            # ---------------- 批次训练循环 ----------------
            n_batches = min(len(source_loader), len(target_loader))  # 取最小批次量，确保源/目标数据同步
            progress_bar = tqdm(range(n_batches), desc=f'Epoch {epoch+1}/{epochs}')
            
            for batch_idx in progress_bar:
                try:
                    # 获取源域和目标域的一个批次数据
                    source_batch = next(iter(source_loader))
                    target_batch = next(iter(target_loader))
                    
                    # 解析数据并移动到计算设备
                    source_inputs, source_targets, _, source_category = [
                        x.float().to(device) if torch.is_tensor(x) else x for x in source_batch
                    ]
                    target_inputs, target_targets, _, target_category = [
                        x.float().to(device) if torch.is_tensor(x) else x for x in target_batch
                    ]
                    
                    # 混合精度训练上下文管理器
                    with torch.cuda.amp.autocast(enabled=True if scaler else False):
                        # 1. 特征提取（使用对应域索引）
                        source_features = model.time_series_encoder(source_inputs, domain_idx=source_domain_idx)
                        target_features = model.time_series_encoder(target_inputs, domain_idx=target_domain_idx)
                        
                        # 2. 执行域对抗训练步骤（核心逻辑）
                        domain_loss, source_domain_pred, target_domain_pred = domain_adversarial_training_step(
                            source_features=source_features,
                            target_features=target_features,
                            domain_discriminator=domain_discriminator,
                            optimizer_disc=optimizer_disc,
                            epoch=epoch,
                            total_epochs=epochs,
                            device=device,
                            projection_layer=projection_layer  # 传入全局投影层处理特征维度
                        )
                        
                        # 3. 任务预测（使用提取的特征预测目标值）
                        source_predictions = model.bilstm_predictor(source_features, source_category)
                        target_predictions = model.bilstm_predictor(target_features, target_category)
                        
                        # 4. 计算任务损失（Huber损失）
                        source_task_loss = huber_loss_fn(source_predictions, source_targets)
                        target_task_loss = huber_loss_fn(target_predictions, target_targets)
                        
                        # 5. 动态平衡源域和目标域任务权重（训练初期侧重源域，后期侧重目标域）
                        src_weight = max(0.2, 1.0 - epoch/epochs)    # 源域权重从1线性衰减到0.2
                        tgt_weight = min(5.0, 1.0 + epoch*4/epochs)   # 目标域权重从1线性增加到5
                        
                        # 6. 计算总损失（任务损失+域对抗损失）
                        total_loss = (
                            source_task_loss * src_weight + 
                            target_task_loss * tgt_weight + 
                            domain_loss * current_lambda  # 域对抗损失乘以动态λ
                        )
                    
                    # 7. 反向传播与优化（主模型和判别器分别优化）
                    optimizer.zero_grad()  # 清空主模型梯度
                    if scaler is not None:
                        # 混合精度训练步骤：缩放损失、反向传播、裁剪梯度、更新参数
                        scaler.scale(total_loss).backward()
                        scaler.unscale_(optimizer)  # 取消梯度缩放，防止溢出
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # 梯度裁剪
                        scaler.step(optimizer)  # 更新参数
                        scaler.update()  # 更新缩放因子
                    else:
                        total_loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                        optimizer.step()
                    
                    # ---------------- 损失统计与进度更新 ----------------
                                        # ---------------- 损失统计与进度更新 ----------------
                    # 保存批次损失，用于下一轮λ调整
                    epoch_source_losses.append(source_task_loss.item())
                    epoch_target_losses.append(target_task_loss.item())
                    epoch_domain_losses.append(domain_loss.item())
                    
                    # 累加统计信息
                    epoch_stats['total_loss'] += total_loss.item()
                    epoch_stats['domain_loss'] += domain_loss.item()
                    epoch_stats['task_loss'] += (source_task_loss.item() + target_task_loss.item())
                    
                    # 更新进度条显示（实时监控损失和λ值）
                    progress_bar.set_postfix({
                        'loss': f"{total_loss.item():.4f}",
                        'src_task': f"{source_task_loss.item():.4f}",
                        'tgt_task': f"{target_task_loss.item():.4f}",
                        'domain': f"{domain_loss.item():.4f}",
                        'λ': f"{current_lambda:.4f}"
                    })
                    
                except Exception as e:
                    logging.warning(f"批次处理出错: {str(e)}")
                    continue  # 跳过当前出错批次，继续下一批次
            
            # ---------------- 学习率更新 ----------------
            scheduler.step()        # 更新主模型学习率
            scheduler_disc.step()   # 更新判别器学习率
            
            # 计算平均损失
            avg_total_loss = epoch_stats['total_loss'] / n_batches
            avg_task_loss = epoch_stats['task_loss'] / n_batches
            avg_domain_loss = epoch_stats['domain_loss'] / n_batches
            
            # ---------------- 模型评估与早停检查 ----------------
            current_rmse = 0
            with torch.no_grad():
                model.eval()  # 设置模型为评估模式
                val_losses = []
                all_predictions = []
                all_targets = []
                
                # 评估前5个批次（加速评估过程，避免全量数据耗时过长）
                for val_batch_idx in range(min(5, len(target_loader))):
                    try:
                        target_batch = next(iter(target_loader))
                        inputs, targets, _, category = [
                            x.float().to(device) if torch.is_tensor(x) else x for x in target_batch
                        ]
                        
                        # 使用目标域索引进行预测（确保模型使用目标域参数）
                        if is_adaptive_model:
                            predictions = model(inputs, category, domain_idx=target_domain_idx)
                        else:
                            predictions = model(inputs, category)
                        
                        # 收集预测结果和真实值
                        all_predictions.append(predictions.detach())
                        all_targets.append(targets.detach())
                        
                        # 计算批次MSE损失
                        val_loss = F.mse_loss(predictions, targets)
                        val_losses.append(val_loss.item())
                    except Exception as e:
                        logging.warning(f"验证批次处理出错: {str(e)}")
                        continue
                
                # 计算整体RMSE（均方根误差，衡量预测精度）
                if all_predictions and all_targets:
                    all_predictions = torch.cat(all_predictions, dim=0)
                    all_targets = torch.cat(all_targets, dim=0)
                    mse = torch.mean((all_predictions - all_targets) ** 2)
                    current_rmse = torch.sqrt(mse).item()
                    history['rmse'].append(current_rmse)
                    
                    logging.info(f"Epoch {epoch+1} 目标域样本评估 - RMSE: {current_rmse:.4f}")
            
            # ---------------- 保存训练历史与早停逻辑 ----------------
            history['total_loss'].append(avg_total_loss)
            history['task_loss'].append(avg_task_loss)
            history['domain_loss'].append(avg_domain_loss)
            
            # 早停判断：基于RMSE是否改进
            if current_rmse > 0 and current_rmse < best_rmse:
                best_rmse = current_rmse
                best_model_state = copy.deepcopy(model.state_dict())
                patience_counter = 0  # 重置计数器
                logging.info(f"发现新的最佳RMSE: {best_rmse:.4f}")
            else:
                patience_counter += 1
                logging.info(f"未改进RMSE，耐心值: {patience_counter}/{early_stopping_patience}")
            
            if patience_counter >= early_stopping_patience:
                logging.info(f"触发早停，在epoch {epoch+1}")
                break  # 停止训练循环
            
            logging.info(
                f"Epoch {epoch+1}/{epochs} - "
                f"Loss: {avg_total_loss:.4f}, "
                f"Task: {avg_task_loss:.4f}, "
                f"Domain: {avg_domain_loss:.4f}, "
                f"LR: {scheduler.get_last_lr()[0]:.6f}, "
                f"λ: {current_lambda:.4f}"
            )
            
    except Exception as e:
        logging.error(f"训练过程出错: {str(e)}")
        import traceback
        traceback.print_exc()
        if best_model_state is not None:
            logging.info("加载最佳模型状态")
            model.load_state_dict(best_model_state)
    
    # 恢复最佳模型
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        logging.info(f"已恢复最佳模型")
    
    # 保存迁移学习模型
    try:
        checkpoint_path = f'models/adapted_ablation_model.pth'  # 修改保存路径以区分
        torch.save({
            'state_dict': model.state_dict(),
            'history': history,
            'best_rmse': best_rmse,
            'best_epoch': epochs - patience_counter,
            'lambda_domain': current_lambda,
            'source_domain_idx': source_domain_idx,
            'target_domain_idx': target_domain_idx,
            'is_adaptive_model': is_adaptive_model,
            'model_type': 'AdaptiveBiLSTMAblation'  # 添加模型类型标识
        }, checkpoint_path)
        logging.info(f"消融实验模型已保存到 {checkpoint_path}")
    except Exception as e:
        logging.error(f"保存模型失败: {str(e)}")
    
    return model, history

In [ ]:
import os
import json
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import bisect
from bilstm import BaselineBiLSTM

# -------------------------- 模型参数设置 --------------------------
batch_size = 32
sequence_length = 24
forecast_horizon = 24
hidden_dim = 64
num_layers = 2
dropout = 0.3
learning_rate = 0.001
weight_decay = 1e-5
epochs = 10
transfer_epochs = 15
input_dim = 6
with open('train_test_labels.json', 'r') as f:
    train_test_labels = json.load(f)
# 定义域索引映射（源域=0，目标域依次为1-5）
DOMAIN_MAPPING = {
    "DO": 1,
    "HO": 2,
    "LI": 3,
    "OF": 4,
    "UL": 5,
    "CC": None
}
# "CC"没有源建筑，所以设置为None,进入贝叶斯域适应的混合域模式。
# -------------------------- 迁移学习配置 --------------------------
data_shortage_scenarios = ['mild', 'heavy', 'extreme']
source_model_path = 'models/adaptive_source_huber_best.pth'  # 更新为适应性模型路径
transfer_results = {}
source_domain_idx = 0  # 源域索引

# -------------------------- 加载所有类别 & 数据维度 --------------------------
print("加载所有类别的训练数据...")
train_loader, test_loader, categories = create_transfer_dataloaders(
    category="DO",
    data_shortage='mild',
    batch_size=batch_size,
    sequence_length=sequence_length,
    forecast_horizon=forecast_horizon
)
print(f"输入形状: torch.Size([32, 6, 24, 6]), 目标形状: torch.Size([32, 6, 24]), 类别形状: torch.Size([32, 5])")
input_dim = 6       # 每个时间步的特征数
num_buildings = 6   # 每个批次中的建筑物数量
category_dim = 6    # 类别数量（不包含CC类别，因为它没有domain_idx）


# 修改加载源域模型的代码部分
if os.path.exists(source_model_path):
    # 加载完整的模型信息字典
    checkpoint = torch.load(source_model_path)
    
    # 创建模型实例 - 使用贝叶斯域自适应模型
    source_model = AdaptiveBiLSTMAblation(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        category_dim=category_dim,
        forecast_horizon=forecast_horizon,
        num_buildings=1,
        num_domains=num_domains,  # 指定域数量
        num_layers=num_layers,
        dropout=dropout
    )
    
    # 如果是完整的检查点格式，需要提取state_dict
    if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
        # 使用strict=False允许部分加载，忽略不匹配的键
        source_model.load_state_dict(checkpoint['state_dict'], strict=False)
        print(f"✅ 从检查点加载自适应源域模型成功 (部分加载): {source_model_path}")
        print(f"   模型保存于第 {checkpoint.get('epoch', 'unknown')} 轮")
        print(f"   验证损失: {checkpoint.get('val_loss', 'unknown')}")
    else:
        # 如果只是普通的state_dict
        source_model.load_state_dict(checkpoint, strict=False)
        print(f"✅ 加载自适应源域模型成功 (部分加载): {source_model_path}")
else:
    print(f"❌ 错误：找不到自适应源域模型 {source_model_path}")
    import sys
    sys.exit(1)

def create_combined_dataloaders(source_category, target_category, data_shortage, batch_size, sequence_length, forecast_horizon):
    """
    构建合并数据加载器（源域：mild，目标域：指定缺失程度）
    对于CC类别，使用所有其他类别的训练数据作为源域
    """
    print(f"为类别 {target_category} 创建数据加载器 (shortage: {data_shortage})...")
    
    try:
        # 正常类别的处理逻辑（非CC类别）
        if train_test_labels[target_category]["train"] is not None:
            # 源域使用完整 mild 数据
            source_train_loader, source_test_loader, _ = create_category_dataloaders(
                category=target_category,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon,
                data_shortage='mild'
            )

            # 目标域使用指定数据缺失场景
            target_train_loader, target_test_loader, _ = create_category_dataloaders(
                category=target_category,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon,
                data_shortage=data_shortage
            )

            # 检查数据加载器是否为None
            if source_train_loader is None or target_train_loader is None:
                print(f"警告：无法创建 {target_category} 的数据加载器")
                return None, None, None, categories

            class SingleBuildingWrapper(Dataset):
                """确保所有样本都只包含一个建筑的数据集包装器"""
                def __init__(self, dataset):
                    self.dataset = dataset

                def __len__(self):
                    return len(self.dataset)

                def __getitem__(self, idx):
                    # 获取原始样本
                    data = self.dataset[idx]
                    if isinstance(data, tuple) and len(data) >= 2:
                        inputs, targets = data[0], data[1]
                        
                        # 检查并确保只使用一个建筑
                        if inputs.shape[0] > 1:
                            inputs = inputs[0:1]  # 只保留第一个建筑
                            targets = targets[0:1]  # 同样只保留第一个建筑的目标
                        
                        # 重建tuple
                        result = (inputs,) + (targets,) + data[2:]
                        return result
                    else:
                        return data

            # 使用包装器处理每个数据集
            source_train_dataset = SingleBuildingWrapper(source_train_loader.dataset)
            source_test_dataset = SingleBuildingWrapper(source_test_loader.dataset)
            target_train_dataset = SingleBuildingWrapper(target_train_loader.dataset)
            target_test_dataset = SingleBuildingWrapper(target_test_loader.dataset)

            class CombinedDataset(Dataset):
                def __init__(self, datasets):
                    self.datasets = datasets
                    self.lengths = [len(ds) for ds in datasets]
                    self.cumulative_lengths = [0]
                    for length in self.lengths:
                        self.cumulative_lengths.append(self.cumulative_lengths[-1] + length)

                def __len__(self):
                    return sum(self.lengths)

                def __getitem__(self, idx):
                    dataset_idx = bisect.bisect_right(self.cumulative_lengths, idx) - 1
                    sample_idx = idx - self.cumulative_lengths[dataset_idx]
                    return self.datasets[dataset_idx][sample_idx]

            # 合并处理后的数据集
            combined_dataset = CombinedDataset([
                source_train_dataset,
                source_test_dataset,
                target_train_dataset
            ])

            # 创建新的数据加载器
            combined_train_loader = DataLoader(
                combined_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )
            
            # 目标测试集也需要使用单建筑格式
            target_test_loader_single = DataLoader(
                target_test_dataset,
                batch_size=batch_size,
                shuffle=False,
                num_workers=0
            )
            
            # 目标训练集也需要使用单建筑格式
            target_train_loader_single = DataLoader(
                target_train_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )

            print(f"成功创建 {target_category} 的数据加载器，共 {len(combined_dataset)} 个样本")
            return combined_train_loader, target_test_loader_single, target_train_loader_single, categories
        
        # CC类别的特殊处理
        else:
            print(f"类别 {target_category} 没有训练数据，使用特殊处理")
            
            # 收集所有其他类别的训练数据作为源域
            all_source_datasets = []
            
            for cat in categories:
                if cat != target_category and train_test_labels[cat]["train"] is not None:
                    # 加载其他类别的训练数据（使用mild数据）
                    cat_train_loader, _, _ = create_category_dataloaders(
                        category=cat,
                        batch_size=batch_size,
                        sequence_length=sequence_length,
                        forecast_horizon=forecast_horizon,
                        data_shortage='mild'  # 源域使用完整数据
                    )
                    if cat_train_loader is not None:
                        all_source_datasets.append(cat_train_loader.dataset)
            
            # 获取CC类别的测试数据（作为目标域）
            _, cc_test_loader, _ = create_category_dataloaders(
                category=target_category,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon,
                data_shortage=data_shortage
            )
            
            if not all_source_datasets or cc_test_loader is None:
                print(f"警告：无法为CC类别创建数据加载器")
                return None, None, None, categories
            
            # 获取CC测试建筑ID
            cc_test_building = train_test_labels[target_category]["test"][0]
            
            # 创建一个简单的单建筑数据集作为训练集
            class EmptyDataset(Dataset):
                def __len__(self):
                    return 10  # 小数据集
                
                def __getitem__(self, idx):
                    # 创建一个兼容的数据项
                    x = torch.zeros((1, sequence_length, input_dim))  # [1建筑, 序列长度, 特征维度]
                    y = torch.zeros((1, forecast_horizon))  # [1建筑, 预测长度]
                    category = target_category
                    category_onehot = torch.zeros(len(categories))
                    category_idx = categories.index(target_category)
                    category_onehot[category_idx] = 1.0
                    return x, y, category, category_onehot
            
            # 使用包装器确保数据格式一致
            class SingleBuildingWrapper(Dataset):
                def __init__(self, dataset):
                    self.dataset = dataset

                def __len__(self):
                    return len(self.dataset)

                def __getitem__(self, idx):
                    data = self.dataset[idx]
                    if isinstance(data, tuple) and len(data) >= 2:
                        inputs, targets = data[0], data[1]
                        if inputs.shape[0] > 1:
                            inputs = inputs[0:1]
                            targets = targets[0:1]
                        result = (inputs,) + (targets,) + data[2:]
                        return result
                    else:
                        return data
            
            # 包装所有源域数据集
            wrapped_source_datasets = [SingleBuildingWrapper(ds) for ds in all_source_datasets]
            
            # 创建空的训练集作为CC的训练数据
            cc_train_dataset = EmptyDataset()
            wrapped_cc_train = SingleBuildingWrapper(cc_train_dataset)
            wrapped_cc_test = SingleBuildingWrapper(cc_test_loader.dataset)
            
            # 合并所有源域数据和CC的训练数据
            class CombinedDataset(Dataset):
                def __init__(self, datasets):
                    self.datasets = datasets
                    self.lengths = [len(ds) for ds in datasets]
                    self.cumulative_lengths = [0]
                    for length in self.lengths:
                        self.cumulative_lengths.append(self.cumulative_lengths[-1] + length)

                def __len__(self):
                    return sum(self.lengths)

                def __getitem__(self, idx):
                    dataset_idx = bisect.bisect_right(self.cumulative_lengths, idx) - 1
                    sample_idx = idx - self.cumulative_lengths[dataset_idx]
                    return self.datasets[dataset_idx][sample_idx]
            
            # combined_train_loader = 所有其他类别的训练数据 + CC的训练时间段数据
            combined_dataset = CombinedDataset(wrapped_source_datasets + [wrapped_cc_train])
            combined_train_loader = DataLoader(
                combined_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )
            
            # target_test_loader_single = CC的测试数据
            target_test_loader_single = DataLoader(
                wrapped_cc_test,
                batch_size=batch_size,
                shuffle=False,
                num_workers=0
            )
            
            # target_train_loader_single = CC的训练时间段数据
            target_train_loader_single = DataLoader(
                wrapped_cc_train,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )
            
            print(f"CC类别数据加载器创建成功:")
            print(f"  - combined_train_loader: {len(combined_dataset)} 样本 (包含所有其他类别 + CC训练时间段)")
            print(f"  - target_test_loader: {len(wrapped_cc_test)} 样本")
            print(f"  - target_train_loader: {len(wrapped_cc_train)} 样本")
            
            return combined_train_loader, target_test_loader_single, target_train_loader_single, categories
    
    except Exception as e:
        print(f"创建数据加载器时出错: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, categories

# -------------------------- 创建基线Huber损失模型 --------------------------
class HuberBaselineBiLSTM(BaselineBiLSTM):
    """
    支持直接输出的基线BiLSTM模型，使用Huber损失训练
    """
    def __init__(self, input_dim, hidden_dim, forecast_horizon, num_buildings=1, num_layers=2, dropout=0.3):
        super().__init__(input_dim, hidden_dim, forecast_horizon, num_buildings, num_layers, dropout)
        
        # 使用单一输出层进行直接预测 - 注意这里使用 hidden_dim * 2 来匹配双向LSTM的输出
        self.fc = torch.nn.Linear(hidden_dim * 2, forecast_horizon)
        
    def forward(self, x, category_onehot=None, domain_idx=None):  # 添加domain_idx参数
        # domain_idx 参数被忽略，仅为了兼容AdaptiveBiLSTM
        
        # x形状: [batch, num_buildings, seq_len, input_dim]
        batch_size, num_buildings, seq_len, _ = x.shape
        
        # 重塑为[batch*num_buildings, seq_len, input_dim]
        x = x.reshape(batch_size * num_buildings, seq_len, -1)
        
        # LSTM层处理
        lstm_out, _ = self.lstm(x)
        
        # 只取最后一个时间步的输出
        out = lstm_out[:, -1, :]
        out = self.dropout(out)
        
        # 直接输出预测值
        predictions = self.fc(out)
        
        # 重塑回[batch, num_buildings, forecast_horizon]
        predictions = predictions.view(batch_size, num_buildings, -1)
        
        return predictions

        
def train_huber_model(model, train_loader, val_loader, epochs=20, lr=0.001, weight_decay=1e-5):
    """训练使用Huber损失的模型"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    
    history = {"train_loss": [], "val_loss": []}
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    
    # 定义Huber损失函数
    huber_loss_fn = HuberLoss(delta=1.0)
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        epoch_loss = 0
        for i, (inputs, targets, *extra) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = huber_loss_fn(predictions, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
            
        train_loss = epoch_loss / len(train_loader)
        history["train_loss"].append(train_loss)
        
        # 验证阶段
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets, *extra in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                predictions = model(inputs)
                loss = huber_loss_fn(predictions, targets)
                val_loss += loss.item()
                
        val_loss = val_loss / len(val_loader)
        history["val_loss"].append(val_loss)
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # 早停
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= 3:
                print(f'Early stopping at epoch {epoch+1}')
                break
    
    # 恢复最佳模型
    if best_model_state:
        model.load_state_dict(best_model_state)
        
    return model, history, best_val_loss

import traceback
import logging
from datetime import datetime, UTC



# 在迁移学习循环之前设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('transfer_learning.log'),
        logging.StreamHandler()
    ]
)

# -------------------------- 主迁移学习循环 --------------------------


source_model_category = 'ALL'
# -------------------------- 主迁移学习循环 --------------------------
for target_category in categories:
    transfer_results[target_category] = {}
    target_domain_idx = DOMAIN_MAPPING[target_category]
    
    for data_shortage in data_shortage_scenarios:
        logging.info(f"\n🚀 [Transfer Learning] Source({source_model_category}) ➜ Target({target_category}), Shortage: {data_shortage}")

        try:
            # 初始化结果字典
            transfer_results[target_category][data_shortage] = {}
            
            # 加载数据
            combined_train_loader, target_test_loader, target_train_loader, _ = create_combined_dataloaders(
                source_category=target_category,
                target_category=target_category,
                data_shortage=data_shortage,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon
            )
            
            # 检查数据加载器
            if combined_train_loader is None or target_test_loader is None:
                print(f"⚠️ 无法为 {target_category}/{data_shortage} 创建数据加载器，跳过")
                continue
            
            # 对于CC类别，仍然可以训练基线模型（使用其训练时间段的数据）
            if target_train_loader is not None:
                # ---------------- 基线模型训练与评估 ----------------
                logging.info(f"📦 Training Huber baseline model for category {target_category}...")
                baseline_model = HuberBaselineBiLSTM(
                    input_dim=input_dim,
                    hidden_dim=hidden_dim,
                    forecast_horizon=forecast_horizon,
                    num_buildings=1,
                    num_layers=num_layers,
                    dropout=dropout
                ).to(device)

                trained_baseline_model, base_history, _ = train_huber_model(
                    model=baseline_model,
                    train_loader=target_train_loader,
                    val_loader=target_test_loader,
                    epochs=transfer_epochs // 2,
                    lr=learning_rate,
                    weight_decay=weight_decay
                )

                # 评估基线模型
                logging.info(f"📊 Evaluating baseline model for category {target_category}...")
                baseline_metrics = evaluate_model(
                    model=trained_baseline_model,
                    test_loader=target_test_loader,
                    model_name=f"Baseline_{target_category}_{data_shortage}",
                    device=device,

                )
                
                # 保存基线模型结果
                transfer_results[target_category][data_shortage]['baseline'] = {
                    'metrics': baseline_metrics,
                    'model_type': "baseline"
                }
            else:
                print(f"⚠️ {target_category} 没有训练数据加载器")
                baseline_metrics = None
                trained_baseline_model = None
            

            
            # ---------------- 迁移学习过程 ----------------
            adapted_model, transfer_history = adapt_to_target_domain_ablation(
                source_model=source_model,
                source_loader=combined_train_loader,
                target_loader=target_test_loader,
                epochs=transfer_epochs,
                lr=learning_rate / 2,
                device=device,
                lambda_domain=0.4,
                early_stopping_patience=3,
                source_domain_idx=source_domain_idx,
                target_domain_idx=target_domain_idx
            )

            # 评估自适应模型
            adaptive_metrics = evaluate_model(
                model=adapted_model,
                test_loader=target_test_loader,
                model_name=f"Adaptive_TL_ALL_to_{target_category}_{data_shortage}",
                baseline_model=trained_baseline_model,
                device=device,
                domain_idx=target_domain_idx
            )

            # ---------------- 结果对比与保存 ----------------
            # 保存基线模型结果
            transfer_results[target_category][data_shortage]['baseline'] = {
                'metrics': baseline_metrics,
                'model_type': "baseline"
            }
            
            # 保存自适应模型结果
            is_adaptive_model = hasattr(adapted_model, 'time_series_encoder')
            model_type = "adaptive" if is_adaptive_model else "chronos"
            transfer_results[target_category][data_shortage]['adaptive'] = {
                'metrics': adaptive_metrics,
                'is_adaptive_model': is_adaptive_model,
                'model_type': model_type,
                'transfer_metrics': {
                    'a_distance': adaptive_metrics.get('a_distance', 'N/A'),
                    'feature_alignment': adaptive_metrics.get('feature_alignment', 'N/A'),
                    'mmd': adaptive_metrics.get('mmd', 'N/A')
                }
            }

            # 打印指标对比表
            print("\n📊 基线模型 vs 自适应模型 指标对比:")
            print_metrics_table([baseline_metrics, adaptive_metrics])
            
            # 计算PIR (概率改进率)
            if 'PIR' in adaptive_metrics and adaptive_metrics['PIR'] is not None:
                if 'RMSD' in baseline_metrics and 'RMSD' in adaptive_metrics and baseline_metrics['RMSD'] > 0:
                    # 直接使用RMSD值计算改进率
                    pir_improvement = (baseline_metrics['RMSD'] - adaptive_metrics['RMSD']) / baseline_metrics['RMSD'] * 100
                    print(f"🚀 自适应模型相比基线的RMSD改进率: {pir_improvement:.2f}%")
                
                # 如果自适应模型有PIR值，单独显示
                print(f"📈 自适应模型的PIR值: {adaptive_metrics['PIR']:.2f}%")
            else:
                print("⚠️ 自适应模型没有PIR值")
            
  

        except Exception as e:
            print(f"❌ Error: {target_category}/{data_shortage} transfer failed: {e}")
            traceback.print_exc()
            continue

print("\n✅ All transfer learning experiments completed")
# 在主循环结束后添加以下代码
print("\n📊 Generating additional visualization charts...")




2025-06-20 20:09:44,194 - INFO - 
🚀 [Transfer Learning] Source(ALL) ➜ Target(DO), Shortage: mild


加载所有类别的训练数据...
加载 mild 场景的电力数据...
准备源域数据...源建筑: ['Hog_lodging_Brian', 'Hog_lodging_Nikki', 'Hog_lodging_Ora', 'Robin_lodging_Celia', 'Robin_lodging_Elmer'] 和目标建筑(训练部分): Robin_lodging_Renea
准备源域电力数据时出错: 'timestamp'
输入形状: torch.Size([32, 6, 24, 6]), 目标形状: torch.Size([32, 6, 24]), 类别形状: torch.Size([32, 5])
✅ 从检查点加载自适应源域模型成功 (部分加载): models/adaptive_source_huber_best.pth
   模型保存于第 13 轮
   验证损失: 0.0035071619473564537
为类别 DO 创建数据加载器 (shortage: mild)...
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
Chronos数据集初始化: 1 个建筑, 6552 个有效样本


2025-06-20 20:09:44,361 - INFO - 📦 Training Huber baseline model for category DO...


Chronos数据集初始化: 5 个建筑, 2136 个有效样本
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
成功创建 DO 的数据加载器，共 10824 个样本
Epoch 1/7, Train Loss: 0.0299, Val Loss: 0.0088
Epoch 2/7, Train Loss: 0.0093, Val Loss: 0.0088
Epoch 3/7, Train Loss: 0.0077, Val Loss: 0.0059
Epoch 4/7, Train Loss: 0.0064, Val Loss: 0.0056
Epoch 5/7, Train Loss: 0.0059, Val Loss: 0.0055
Epoch 6/7, Train Loss: 0.0057, Val Loss: 0.0055


2025-06-20 20:12:24,408 - INFO - 📊 Evaluating baseline model for category DO...


...

形状信息:
预测: (157248,)
目标: (157248,)
下界: (157248,)
上界: (157248,)
不确定性: (157248,)


2025-06-20 23:20:34,981 - INFO - 检测到3D特征，进行展平: source_features.shape=torch.Size([6552, 24, 64])
2025-06-20 23:20:34,984 - INFO - 展平后: source_features.shape=torch.Size([6552, 64]), target_features.shape=torch.Size([6552, 64])
2025-06-20 23:20:34,985 - WARNING - 特征数量过大，进行采样: source_features.shape=torch.Size([6552, 64]), target_features.shape=torch.Size([6552, 64])
2025-06-20 23:20:34,986 - INFO - 采样后: source_features.shape=torch.Size([5000, 64]), target_features.shape=torch.Size([5000, 64])
2025-06-20 23:20:35,283 - INFO - 
🚀 [Transfer Learning] Source(ALL) ➜ Target(UL), Shortage: heavy



Adaptive_TL_ALL_to_UL_mild 评估报告:
RMSD: 0.09914001979447762
MAPE: 21.8519229888916%
R²: -0.12335669994354248
KL散度: N/A

迁移学习评估:
域间距离 (A-distance): 1.336400032043457
特征对齐质量: 0.8404147028923035
MMD: 0.291778564453125

不确定性评估:
预测区间覆盖率(PICP): 49.24% (目标95%)
平均预测区间宽度(NMPIW): 0.19165505468845367
校准误差: 45.76%
平均不确定性(标准差): 0.028478268533945084

相比基线的改进率:
RMSD改进率(PIR): -45.05%

📊 基线模型 vs 自适应模型 指标对比:
Model           MAPE(%)    RMSD       CC         R2         PIR(%)    
----------------------------------------------------------------------
Baseline_UL_mild 19.74      0.07       0.75       0.46       N/A       
Adaptive_TL_ALL_to_UL_mild 21.85      0.10       0.18       -0.12      -45.05    

不确定性评估指标:
Model           PICP(%)    校准误差(%)         区间宽度            UQS       
----------------------------------------------------------------------
Baseline_UL_mild 68.91      26.09           0.2385          0.4396    
Adaptive_TL_ALL_to_UL_mild 49.24      45.76           0.1917          0.3085    
🚀 自适应模

2025-06-20 23:20:35,443 - INFO - 📦 Training Huber baseline model for category UL...


Chronos数据集初始化: 5 个建筑, 624 个有效样本
Chronos数据集初始化: 1 个建筑, 8064 个有效样本
成功创建 UL 的数据加载器，共 9312 个样本
Epoch 1/7, Train Loss: 0.0823, Val Loss: 0.0561
Epoch 2/7, Train Loss: 0.0213, Val Loss: 0.0308
Epoch 3/7, Train Loss: 0.0182, Val Loss: 0.0270


In [20]:
# ======================== 保存所有实验结果 ========================
print("💾 正在保存所有实验结果...")

# 创建保存目录
os.makedirs('results', exist_ok=True)
os.makedirs('results/ablation', exist_ok=True)
os.makedirs('results/visualizations/ablation', exist_ok=True)


# 2. 导出关键指标为CSV（便于分析）
try:
    import pandas as pd
    
    # 准备CSV数据
    csv_data = []
    for category in transfer_results:
        for shortage in transfer_results[category]:
            # 基线模型指标
            if 'baseline' in transfer_results[category][shortage] and transfer_results[category][shortage]['baseline']['metrics']:
                baseline = transfer_results[category][shortage]['baseline']
                baseline_metrics = baseline['metrics']
                
                row = {
                    'Category': category,
                    'Data_Shortage': shortage,
                    'Model': 'Baseline',
                    'Model_Type': baseline['model_type']
                }
                
                # 添加基本指标
                for key in ['RMSE', 'RMSD', 'CC', 'MAPE', 'R2', 'PICP', 'NMPIW', 'calibration_error', 'UQS']:
                    if key in baseline_metrics:
                        # 处理不同类型的值
                        value = baseline_metrics[key]
                        if isinstance(value, (np.ndarray, torch.Tensor)):
                            if hasattr(value, 'item'):
                                value = value.item()
                            else:
                                value = float(value)
                        row[key] = value
                
                csv_data.append(row)
            
            # 自适应模型指标
            if 'adaptive' in transfer_results[category][shortage] and transfer_results[category][shortage]['adaptive']['metrics']:
                adaptive = transfer_results[category][shortage]['adaptive']
                adaptive_metrics = adaptive['metrics']
                
                row = {
                    'Category': category,
                    'Data_Shortage': shortage,
                    'Model': 'Adaptive',
                    'Model_Type': adaptive['model_type'],
                    'Is_Adaptive': adaptive['is_adaptive_model']
                }
                
                # 添加基本指标
                for key in ['RMSE', 'RMSD', 'MAE', 'MAPE', 'R2', 'PICP', 'NMPIW', 'calibration_error', 'UQS']:
                    if key in adaptive_metrics:
                        # 处理不同类型的值
                        value = adaptive_metrics[key]
                        if isinstance(value, (np.ndarray, torch.Tensor)):
                            if hasattr(value, 'item'):
                                value = value.item()
                            else:
                                value = float(value)
                        row[key] = value
                
                # 添加迁移学习指标
                if 'transfer_metrics' in adaptive:
                    for key in ['a_distance', 'feature_alignment', 'mmd']:
                        if key in adaptive['transfer_metrics']:
                            value = adaptive['transfer_metrics'][key]
                            if isinstance(value, (np.ndarray, torch.Tensor)):
                                if hasattr(value, 'item'):
                                    value = value.item()
                                else:
                                    value = float(value)
                            row[key] = value
                
                csv_data.append(row)
    
    # 创建DataFrame并保存
    results_df = pd.DataFrame(csv_data)
    results_df.to_csv('results/ablation/transfer_learning_metrics.csv', index=False)
    print("✅ 保存指标摘要到 results/ablation/transfer_learning_metrics.csv")
    
    # 3. 计算并保存改进率
    improvement_data = []
    for category in transfer_results:
        for shortage in transfer_results[category]:
            if ('baseline' in transfer_results[category][shortage] and 
                'adaptive' in transfer_results[category][shortage] and
                transfer_results[category][shortage]['baseline']['metrics'] and 
                transfer_results[category][shortage]['adaptive']['metrics']):
                
                baseline_metrics = transfer_results[category][shortage]['baseline']['metrics']
                adaptive_metrics = transfer_results[category][shortage]['adaptive']['metrics']
                
                # 确保两个模型都有RMSE指标
                if 'RMSE' in baseline_metrics and 'RMSE' in adaptive_metrics:
                    baseline_rmse = baseline_metrics['RMSE']
                    adaptive_rmse = adaptive_metrics['RMSE']
                    
                    # 计算RMSE改进率
                    if isinstance(baseline_rmse, (np.ndarray, torch.Tensor)):
                        baseline_rmse = float(baseline_rmse)
                    if isinstance(adaptive_rmse, (np.ndarray, torch.Tensor)):
                        adaptive_rmse = float(adaptive_rmse)
                    
                    if baseline_rmse > 0:
                        rmse_improvement = ((baseline_rmse - adaptive_rmse) / baseline_rmse) * 100
                    else:
                        rmse_improvement = 0
                    
                    # 提取迁移学习指标
                    a_distance = None
                    mmd = None
                    feature_alignment = None
                    
                    if 'transfer_metrics' in transfer_results[category][shortage]['adaptive']:
                        tm = transfer_results[category][shortage]['adaptive']['transfer_metrics']
                        a_distance = tm.get('a_distance')
                        mmd = tm.get('mmd')
                        feature_alignment = tm.get('feature_alignment')
                    
                    improvement_data.append({
                        'Category': category,
                        'Data_Shortage': shortage,
                        'Baseline_RMSE': baseline_rmse,
                        'Adaptive_RMSE': adaptive_rmse,
                        'RMSE_Improvement': rmse_improvement,
                        'A_Distance': a_distance,
                        'MMD': mmd,
                        'Feature_Alignment': feature_alignment
                    })
    
    # 创建DataFrame并保存
    if improvement_data:
        improvement_df = pd.DataFrame(improvement_data)
        improvement_df.to_csv('results/ablation/improvement_metrics.csv', index=False)
        print("✅ 保存改进率指标到 results/ablation/improvement_metrics.csv")
    
except Exception as e:
    print(f"❌ 导出CSV文件失败: {str(e)}")
    traceback.print_exc()

# 4. 生成汇总可视化
try:
    # 如果我们已经创建了DataFrame，直接使用它
    if 'results_df' in locals() and len(results_df) > 0:
        plt.figure(figsize=(15, 10))
        
        # RMSE对比图
        plt.subplot(2, 2, 1)
        baseline_df = results_df[results_df['Model'] == 'Baseline']
        adaptive_df = results_df[results_df['Model'] == 'Adaptive']
        
        categories = results_df['Category'].unique()
        x = np.arange(len(categories))
        width = 0.35
        
        # 绘制RMSE条形图
        if 'RMSE' in results_df.columns:
            baseline_rmse = [baseline_df[baseline_df['Category'] == cat]['RMSE'].mean() 
                             for cat in categories]
            adaptive_rmse = [adaptive_df[adaptive_df['Category'] == cat]['RMSE'].mean() 
                             for cat in categories]
            
            plt.bar(x - width/2, baseline_rmse, width, label='Baseline')
            plt.bar(x + width/2, adaptive_rmse, width, label='Adaptive')
            plt.xlabel('Building Category')
            plt.ylabel('RMSE')
            plt.title('RMSE Comparison: Baseline vs Adaptive')
            plt.xticks(x, categories)
            plt.legend()
            
        # 绘制MAPE对比图
        plt.subplot(2, 2, 2)
        if 'MAPE' in results_df.columns:
            baseline_mape = [baseline_df[baseline_df['Category'] == cat]['MAPE'].mean() 
                             for cat in categories]
            adaptive_mape = [adaptive_df[adaptive_df['Category'] == cat]['MAPE'].mean() 
                             for cat in categories]
            
            plt.bar(x - width/2, baseline_mape, width, label='Baseline')
            plt.bar(x + width/2, adaptive_mape, width, label='Adaptive')
            plt.xlabel('Building Category')
            plt.ylabel('MAPE (%)')
            plt.title('MAPE Comparison: Baseline vs Adaptive')
            plt.xticks(x, categories)
            plt.legend()
        
        # 绘制迁移学习指标
        plt.subplot(2, 2, 3)
        if 'improvement_df' in locals() and len(improvement_df) > 0:
            for cat in categories:
                cat_data = improvement_df[improvement_df['Category'] == cat]
                if len(cat_data) > 0:
                    plt.scatter(cat_data['Feature_Alignment'], cat_data['RMSE_Improvement'], 
                               label=cat, s=50, alpha=0.7)
            
            plt.xlabel('Feature Alignment')
            plt.ylabel('RMSE Improvement (%)')
            plt.title('Feature Alignment vs RMSE Improvement')
            plt.grid(alpha=0.3)
            plt.legend()
        
        # 绘制数据短缺场景的影响
        plt.subplot(2, 2, 4)
        if 'improvement_df' in locals() and len(improvement_df) > 0:
            shortages = improvement_df['Data_Shortage'].unique()
            for shortage in shortages:
                shortage_data = improvement_df[improvement_df['Data_Shortage'] == shortage]
                plt.boxplot([shortage_data['RMSE_Improvement']], positions=[list(shortages).index(shortage)],
                           labels=[shortage], widths=0.5)
            
            plt.ylabel('RMSE Improvement (%)')
            plt.title('Impact of Data Shortage on Improvement')
            plt.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('results/visualizations/ablation/summary_comparison.png', dpi=300)
        plt.close()
        print("✅ 保存汇总可视化到 results/visualizations/ablation/summary_comparison.png")
        
except Exception as e:
    print(f"❌ 生成汇总可视化失败: {str(e)}")
    traceback.print_exc()

print("✅ 结果保存完成!")

💾 正在保存所有实验结果...
✅ 保存指标摘要到 results/ablation/transfer_learning_metrics.csv
✅ 保存汇总可视化到 results/visualizations/ablation/summary_comparison.png
✅ 结果保存完成!
